## Reading Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
forest = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Forest_Data/full_forest')
forest.head()

,Unnamed: 0,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area_01,Wilderness_Area_02,Wilderness_Area_03,Wilderness_Area_04,Soil_Type_01,Soil_Type_02,Soil_Type_03,Soil_Type_04,Soil_Type_05,Soil_Type_06,Soil_Type_07,Soil_Type_08,Soil_Type_09,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_19,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type,Wilderness,Soil
0,0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,1,29
1,1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,1,29
2,2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,12
3,3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,30
4,4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5,1,29


Removing index column and dummy columns of soil_types and wilderness_areas and storing it in a dataframe **'forest_a'** 

In [ ]:
forest = forest.drop(columns = forest.columns[:1])
forest_a = forest.drop(columns = forest.columns[10:54])

Converting soil, wilderness and cover_type columns to category

In [ ]:
for i in forest.columns[::-1][:3]:
  forest_a[i] = forest_a[i].astype('category')

In [ ]:
forest_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           581012 non-null  int64   
 1   Aspect                              581012 non-null  int64   
 2   Slope                               581012 non-null  int64   
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64   
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64   
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64   
 6   Hillshade_9am                       581012 non-null  int64   
 7   Hillshade_Noon                      581012 non-null  int64   
 8   Hillshade_3pm                       581012 non-null  int64   
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64   
 10  Cover_Type                          581012 non-null  category
 11  Wilderness   

## Modelling on Original Data

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data to train and test in 7:3 ratio

In [ ]:
x = forest_a.drop(columns = ['Cover_Type'])
y = forest_a['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9301390678324498
Testing Score: 0.9029396915733431


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.90      0.90      0.90     63514
           2       0.91      0.92      0.92     85045
           3       0.90      0.90      0.90     10747
           4       0.84      0.82      0.83       804
           5       0.81      0.73      0.76      2844
           6       0.83      0.82      0.82      5235
           7       0.93      0.91      0.92      6115

    accuracy                           0.90    174304
   macro avg       0.87      0.86      0.86    174304
weighted avg       0.90      0.90      0.90    174304

[[56973  6108    11     0    64     9   349]
 [ 5937 78137   286     5   382   251    47]
 [   11   352  9685    92    20   587     0]
 [    0     2   108   660     0    34     0]
 [   58   647    51     0  2069    19     0]
 [    9   247   641    28    32  4278     0]
 [  462    69     0     0     0     0  5584]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.429758
1,Aspect,0.016254
2,Slope,0.009723
3,Horizontal_Distance_To_Hydrology,0.048272
4,Vertical_Distance_To_Hydrology,0.032472
5,Horizontal_Distance_To_Roadways,0.120886
6,Hillshade_9am,0.025126
7,Hillshade_Noon,0.024433
8,Hillshade_3pm,0.013320
9,Horizontal_Distance_To_Fire_Points,0.121630


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9509943251669503
Testing Score: 0.9273051679823756


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.94      0.91      0.92     63514
           2       0.92      0.96      0.94     85045
           3       0.91      0.95      0.93     10747
           4       0.91      0.82      0.86       804
           5       0.93      0.64      0.76      2844
           6       0.91      0.84      0.87      5235
           7       0.97      0.91      0.94      6115

    accuracy                           0.93    174304
   macro avg       0.93      0.86      0.89    174304
weighted avg       0.93      0.93      0.93    174304

[[57522  5805     1     0    18    12   156]
 [ 3042 81495   230     4   121   127    26]
 [    3   267 10163    35     1   278     0]
 [    0     0   129   659     0    16     0]
 [   42   906    48     0  1829    19     0]
 [    2   252   539    24     2  4416     0]
 [  523    43     0     0     0     0  5549]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.309273
1,Aspect,0.029003
2,Slope,0.019739
3,Horizontal_Distance_To_Hydrology,0.043881
4,Vertical_Distance_To_Hydrology,0.037401
5,Horizontal_Distance_To_Roadways,0.106711
6,Hillshade_9am,0.025773
7,Hillshade_Noon,0.026322
8,Hillshade_3pm,0.021980
9,Horizontal_Distance_To_Fire_Points,0.097453


### KNN


*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_a.drop(columns= ['Cover_Type','Soil','Wilderness'])

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_a[['Soil','Wilderness']]], axis = 1)
y_ = forest_a['Cover_Type']
x_.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Soil,Wilderness
0,-1.297805,-0.935157,-1.482820,-0.053767,-0.796273,-1.180146,0.330743,0.439143,0.142960,3.246283,29,1
1,-1.319235,-0.890480,-1.616363,-0.270188,-0.899197,-1.257106,0.293388,0.590899,0.221342,3.205504,29,1
2,-0.554907,-0.148836,-0.681563,-0.006719,0.318742,0.532212,0.816364,0.742654,-0.196691,3.126965,12,1
3,-0.622768,-0.005869,0.520322,-0.129044,1.227908,0.474492,0.965786,0.742654,-0.536343,3.194931,30,1
4,-1.301377,-0.988770,-1.616363,-0.547771,-0.813427,-1.256464,0.293388,0.540313,0.195215,3.165479,29,1


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9372647787773086


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.94      0.93      0.94     63514
           2       0.94      0.95      0.95     85045
           3       0.91      0.92      0.92     10747
           4       0.88      0.82      0.85       804
           5       0.85      0.80      0.82      2844
           6       0.83      0.85      0.84      5235
           7       0.95      0.95      0.95      6115

    accuracy                           0.94    174304
   macro avg       0.90      0.89      0.90    174304
weighted avg       0.94      0.94      0.94    174304

[[59348  3862     2     0    42    17   243]
 [ 3213 80973   246     2   321   252    38]
 [   10   219  9885    61    28   544     0]
 [    0     1    98   661     0    44     0]
 [   56   455    36     0  2275    22     0]
 [    7   203   538    28    16  4443     0]
 [  297    34     0     0     0     0  5784]]


### Ensemble Techniques on Original Data

In [ ]:
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)

base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)

base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Accuracy:',bgcl_DT.score(x_train,y_train))
print('Testing Accuracy:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.9380907186482685
Training Accuracy: 0.963979070979671
Testing Accuracy: 0.9411373232972279


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.93      0.95      0.94     10747
           4       0.91      0.85      0.88       804
           5       0.92      0.74      0.82      2844
           6       0.91      0.88      0.90      5235
           7       0.96      0.93      0.94      6115

    accuracy                           0.94    174304
   macro avg       0.93      0.89      0.91    174304
weighted avg       0.94      0.94      0.94    174304

[[59092  4201     5     0    31     4   181]
 [ 2879 81703   179     3   143   103    35]
 [    4   216 10182    43    16   286     0]
 [    0     0   101   684     0    19     0]
 [   38   631    49     0  2106    20     0]
 [    7   172   412    22     3  4619     0]
 [  413    44     0     0     0     0  5658]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Accuracy:',bgcl_RF.score(x_train,y_train))
print('Testing Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9138300697306175
Training Accuracy: 0.9338247587950077
Testing Accuracy: 0.9147466495318524


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.93      0.89      0.91     63514
           2       0.90      0.95      0.93     85045
           3       0.90      0.94      0.92     10747
           4       0.91      0.79      0.85       804
           5       0.93      0.56      0.70      2844
           6       0.90      0.80      0.84      5235
           7       0.96      0.88      0.92      6115

    accuracy                           0.91    174304
   macro avg       0.92      0.83      0.87    174304
weighted avg       0.92      0.91      0.91    174304

[[56647  6659     2     0    14     9   183]
 [ 3565 80922   278     4    94   152    30]
 [    3   342 10070    37     2   293     0]
 [    0     0   156   636     0    12     0]
 [   40  1148    49     0  1594    13     0]
 [    0   352   680    24     1  4178     0]
 [  676    42     0     0     0     0  5397]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Accuracy:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Accuracy:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9361458343578194
Training Accuracy: 0.9998303451124639
Testing Accuracy: 0.936857444464843


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.94      0.93      0.94     63514
           2       0.94      0.95      0.95     85045
           3       0.91      0.92      0.92     10747
           4       0.88      0.81      0.85       804
           5       0.85      0.79      0.82      2844
           6       0.83      0.84      0.84      5235
           7       0.95      0.94      0.95      6115

    accuracy                           0.94    174304
   macro avg       0.90      0.89      0.89    174304
weighted avg       0.94      0.94      0.94    174304

[[59243  3973     1     0    37    18   242]
 [ 3119 81072   248     3   310   252    41]
 [   10   238  9883    58    24   534     0]
 [    0     0   101   654     0    49     0]
 [   59   461    39     0  2260    24     1]
 [    8   223   552    28    14  4410     0]
 [  303    36     0     0     0     0  5776]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('Training Accuracy:',abcl_DT.score(x_train,y_train))
print('Testing Accuracy:',abcl_DT.score(x_test,y_test))

Training Accuracy: 0.9999704948021676
Testing Accuracy: 0.9539769597943822


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.96      0.94      0.95     63514
           2       0.95      0.97      0.96     85045
           3       0.95      0.97      0.96     10747
           4       0.95      0.85      0.90       804
           5       0.96      0.77      0.85      2844
           6       0.94      0.91      0.93      5235
           7       0.98      0.94      0.96      6115

    accuracy                           0.95    174304
   macro avg       0.96      0.91      0.93    174304
weighted avg       0.95      0.95      0.95    174304

[[59962  3447     1     0    14     1    89]
 [ 2186 82565   129     2    75    74    14]
 [    3   160 10387    17     5   175     0]
 [    0     0    98   685     0    21     0]
 [   29   589    34     0  2177    14     1]
 [    0   158   305    17     3  4752     0]
 [  334    27     0     0     0     0  5754]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('Training Accuracy:',abcl_RF.score(x_train,y_train))
print('Testing Accuracy:',abcl_RF.score(x_test,y_test))

Training Accuracy: 0.7101483127944372
Testing Accuracy: 0.6796688544152745


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.55      1.00      0.71     63514
           2       0.96      0.46      0.62     85045
           3       0.85      0.93      0.89     10747
           4       0.89      0.62      0.73       804
           5       0.96      0.40      0.57      2844
           6       0.86      0.70      0.77      5235
           7       1.00      0.09      0.17      6115

    accuracy                           0.68    174304
   macro avg       0.87      0.60      0.64    174304
weighted avg       0.80      0.68      0.66    174304

[[63254   260     0     0     0     0     0]
 [45162 39411   276     2    36   158     0]
 [   58   300  9963    35     5   386     0]
 [    0     0   272   501     0    31     0]
 [  745   911    34     0  1143    11     0]
 [   92   270  1204    23     1  3645     0]
 [ 5561     2     0     0     0     0   552]]


### Gradient Boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 0.9986009618694494
Testing Score: 0.966518266935928


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.97      0.97      0.97     63514
           2       0.97      0.98      0.97     85045
           3       0.95      0.94      0.94     10747
           4       0.67      0.75      0.71       804
           5       0.93      0.84      0.88      2844
           6       0.92      0.91      0.91      5235
           7       0.98      0.96      0.97      6115

    accuracy                           0.97    174304
   macro avg       0.91      0.91      0.91    174304
weighted avg       0.97      0.97      0.97    174304

[[61469  1897     2     0    21     6   119]
 [ 1344 83327   111    11   143    89    20]
 [   11   164 10066   221    13   272     0]
 [    1    12   143   606     0    41     1]
 [   27   374    34     0  2395    14     0]
 [    3   129   290    61     1  4751     0]
 [  225    33     0     3     0     0  5854]]


## Feature Engineering

Min scaling 'vertical_distance_to_hydrology' column as it has got some negative values

In [ ]:
forest_a['Vertical_Scaled'] = forest_a['Vertical_Distance_To_Hydrology'] - min(forest_a['Vertical_Distance_To_Hydrology'])

Calculating Euclidean distance to hydrology. As water being the necessity for a vegetation to grow.

In [ ]:
forest_a['Hydr_Dist'] = (forest_a['Horizontal_Distance_To_Hydrology']**2 + forest_a['Vertical_Distance_To_Hydrology']**2)**0.5

Taking average of all the Hillshades

In [ ]:
forest_a['Hillshades'] = (forest_a['Hillshade_3pm'] + forest_a['Hillshade_Noon'] + forest_a['Hillshade_9am'])/3

Taking Linear Combination of 'Horizontal_distance_to_roadways and 'Horizontal_distance_to_Firepoints'.

In [ ]:
forest_a['Fire_Road'] = forest_a['Horizontal_Distance_To_Fire_Points'] + forest_a['Horizontal_Distance_To_Roadways']

Taking average of all the horizontal distances columns. Fire_points, hydrology and roadways being the basic amenities affecting any vegetation.

In [ ]:
forest_a['Avg_Horizontal'] = (forest_a['Horizontal_Distance_To_Fire_Points'] + 
                              forest_a['Horizontal_Distance_To_Hydrology'] + 
                              forest_a['Horizontal_Distance_To_Roadways'])/3

Taking the average of 'Horizotal_Distance_to_Fire_points' and 'Horizontal_distance_to_hydrology' column.</b>

In [ ]:
forest_a['Fire_Hydr'] = (forest_a['Horizontal_Distance_To_Fire_Points'] + forest_a['Horizontal_Distance_To_Hydrology'])/2

Calculating at what steepness the particular cover_type is located from hydrology

In [ ]:
forest_a['Hydr_Slope'] = forest_a['Vertical_Distance_To_Hydrology']/forest_a['Horizontal_Distance_To_Hydrology']

converting the slope values into degrees and replacing all the null values with 0

In [ ]:
from math import degrees
a = []
for i in forest_a['Hydr_Slope']:
  a.append(degrees(np.arctan(i)))

b = pd.DataFrame({'a':a})
forest_a['Hydr_Slope'] = np.where(b['a'].isnull(),0,b['a'])
forest_a['Hydr_Slope'].isnull().sum()  

Making a list of all numerical Columns and storing it in a variable 'quant_col'

In [ ]:
quant_col = list(forest_a.columns[:10])
quant_col.extend(forest_a.columns[::-1][:7])
quant_col

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Hydr_Slope',
 'Fire_Hydr',
 'Avg_Horizontal',
 'Fire_Road',
 'Hillshades',
 'Hydr_Dist',
 'Vertical_Scaled']

In [ ]:
forest_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 20 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           581012 non-null  int64   
 1   Aspect                              581012 non-null  int64   
 2   Slope                               581012 non-null  int64   
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64   
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64   
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64   
 6   Hillshade_9am                       581012 non-null  int64   
 7   Hillshade_Noon                      581012 non-null  int64   
 8   Hillshade_3pm                       581012 non-null  int64   
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64   
 10  Cover_Type                          581012 non-null  category
 11  Wilderness   

## Modelling after Feature Engineering

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data to train and test in 7:3 ratio

In [ ]:
x = forest_a.drop(columns = ['Cover_Type','Vertical_Scaled'])
y = forest_a['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9350861060023408
Testing Score: 0.9073802092895171


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.90      0.90      0.90     63514
           2       0.92      0.92      0.92     85045
           3       0.90      0.91      0.91     10747
           4       0.83      0.80      0.82       804
           5       0.79      0.76      0.78      2844
           6       0.85      0.82      0.83      5235
           7       0.95      0.91      0.93      6115

    accuracy                           0.91    174304
   macro avg       0.88      0.86      0.87    174304
weighted avg       0.91      0.91      0.91    174304

[[57123  6035     8     0    82    10   256]
 [ 5470 78565   271     2   461   228    48]
 [    5   347  9822    83    20   470     0]
 [    0     4   125   641     0    34     0]
 [   55   559    45     0  2168    17     0]
 [   10   243   642    42    15  4283     0]
 [  511    45     1     0     0     0  5558]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.423029
1,Aspect,0.013222
2,Slope,0.006725
3,Horizontal_Distance_To_Hydrology,0.013419
4,Vertical_Distance_To_Hydrology,0.018751
5,Horizontal_Distance_To_Roadways,0.075299
6,Hillshade_9am,0.022921
7,Hillshade_Noon,0.018160
8,Hillshade_3pm,0.009535
9,Horizontal_Distance_To_Fire_Points,0.054952


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9648420980162672
Testing Score: 0.9435124839361116


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.97      0.95     85045
           3       0.93      0.96      0.94     10747
           4       0.91      0.85      0.88       804
           5       0.94      0.72      0.81      2844
           6       0.93      0.87      0.90      5235
           7       0.97      0.93      0.95      6115

    accuracy                           0.94    174304
   macro avg       0.94      0.89      0.91    174304
weighted avg       0.94      0.94      0.94    174304

[[58887  4481     1     0    17     3   125]
 [ 2357 82272   172     4   113   104    23]
 [    1   185 10318    39     5   199     0]
 [    0     0   106   684     0    14     0]
 [   35   695    53     0  2050    11     0]
 [    0   202   441    26     4  4562     0]
 [  394    36     0     0     0     0  5685]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.280369
1,Aspect,0.021407
2,Slope,0.011833
3,Horizontal_Distance_To_Hydrology,0.021981
4,Vertical_Distance_To_Hydrology,0.021473
5,Horizontal_Distance_To_Roadways,0.065575
6,Hillshade_9am,0.020614
7,Hillshade_Noon,0.016448
8,Hillshade_3pm,0.015824
9,Horizontal_Distance_To_Fire_Points,0.049737


### KNN

*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio


In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_a.drop(columns= ['Cover_Type','Soil','Wilderness','Vertical_Scaled'])
y_ = forest_a['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_a[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Hydr_Dist,Hillshades,Fire_Road,Avg_Horizontal,Fire_Hydr,Hydr_Slope,Soil,Wilderness
0,-1.297805,-0.935157,-1.482820,-0.053767,-0.796273,-1.180146,0.330743,0.439143,0.142960,3.246283,-0.083233,0.530152,1.043224,1.027143,3.171097,-1.041587,29,1
1,-1.319235,-0.890480,-1.616363,-0.270188,-0.899197,-1.257106,0.293388,0.590899,0.221342,3.205504,-0.294777,0.645367,0.969391,0.934802,3.097132,-1.228740,29,1
2,-0.554907,-0.148836,-0.681563,-0.006719,0.318742,0.532212,0.816364,0.742654,-0.196691,3.126965,-0.001362,0.668410,2.109122,2.085710,3.061629,0.532279,12,1
3,-0.622768,-0.005869,0.520322,-0.129044,1.227908,0.474492,0.965786,0.742654,-0.536343,3.194931,-0.031466,0.461022,2.109122,2.074797,3.108966,1.959275,30,1
4,-1.301377,-0.988770,-1.616363,-0.547771,-0.813427,-1.256464,0.293388,0.540313,0.195215,3.165479,-0.566983,0.599281,0.947327,0.888212,3.014291,-1.084819,29,1


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9477120433266017


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.95      0.95      0.95     63514
           2       0.95      0.96      0.96     85045
           3       0.93      0.93      0.93     10747
           4       0.87      0.86      0.87       804
           5       0.86      0.84      0.85      2844
           6       0.87      0.88      0.87      5235
           7       0.96      0.95      0.96      6115

    accuracy                           0.95    174304
   macro avg       0.91      0.91      0.91    174304
weighted avg       0.95      0.95      0.95    174304

[[60038  3189     2     0    45    20   220]
 [ 2683 81625   205     4   292   199    37]
 [    7   202 10009    65    32   432     0]
 [    0     2    72   695     0    35     0]
 [   50   358    29     0  2383    24     0]
 [    8   165   398    38    12  4614     0]
 [  256    33     0     0     0     0  5826]]


### Ensemble Techniques on data after feature enineering

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Accuracy:',bgcl_DT.score(x_train,y_train))
print('Testing Accuracy:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.9421181781523845
Training Accuracy: 0.9677286898708656
Testing Accuracy: 0.9455089957774921


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.94      0.96      0.95     10747
           4       0.90      0.85      0.88       804
           5       0.91      0.77      0.84      2844
           6       0.93      0.89      0.91      5235
           7       0.97      0.93      0.95      6115

    accuracy                           0.95    174304
   macro avg       0.94      0.90      0.92    174304
weighted avg       0.95      0.95      0.95    174304

[[59374  3961     1     0    30     1   147]
 [ 2679 81905   161     4   160   103    33]
 [    2   188 10288    44    10   215     0]
 [    0     0   102   683     0    19     0]
 [   36   543    48     0  2195    22     0]
 [    4   144   398    25     4  4660     0]
 [  383    31     0     0     0     0  5701]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Accuracy:',bgcl_RF.score(x_train,y_train))
print('Testing Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9302816762886396
Training Accuracy: 0.949125662637568
Testing Accuracy: 0.930621213512025


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.91      0.93     63514
           2       0.92      0.96      0.94     85045
           3       0.91      0.95      0.93     10747
           4       0.90      0.80      0.85       804
           5       0.93      0.64      0.76      2844
           6       0.92      0.84      0.88      5235
           7       0.97      0.90      0.93      6115

    accuracy                           0.93    174304
   macro avg       0.93      0.86      0.89    174304
weighted avg       0.93      0.93      0.93    174304

[[58011  5328     1     0    16     8   150]
 [ 2924 81633   224     5   108   127    24]
 [    3   274 10192    41     2   235     0]
 [    0     0   144   644     0    16     0]
 [   40   925    49     0  1818    12     0]
 [    0   272   541    29     1  4392     0]
 [  555    39     0     0     0     0  5521]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Accuracy:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Accuracy:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9453268684166527
Training Accuracy: 0.9998746029092125
Testing Accuracy: 0.9468055810537911


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.95      0.94      0.95     63514
           2       0.95      0.96      0.96     85045
           3       0.93      0.93      0.93     10747
           4       0.86      0.85      0.86       804
           5       0.87      0.83      0.85      2844
           6       0.86      0.88      0.87      5235
           7       0.96      0.95      0.95      6115

    accuracy                           0.95    174304
   macro avg       0.91      0.91      0.91    174304
weighted avg       0.95      0.95      0.95    174304

[[59949  3292     1     0    45    17   210]
 [ 2657 81644   211     6   280   207    40]
 [    8   202 10003    64    26   444     0]
 [    0     1    87   683     0    33     0]
 [   54   374    33     0  2358    24     1]
 [    8   188   408    37    11  4583     0]
 [  269    34     0     0     0     0  5812]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('Training Accuracy:',abcl_DT.score(x_train,y_train))
print('Testing Accuracy:',abcl_DT.score(x_test,y_test))

Training Accuracy: 0.9999631185027096
Testing Accuracy: 0.9589166054709014


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.96      0.95      0.96     63514
           2       0.95      0.97      0.96     85045
           3       0.96      0.97      0.96     10747
           4       0.94      0.86      0.90       804
           5       0.96      0.78      0.86      2844
           6       0.95      0.92      0.94      5235
           7       0.99      0.95      0.97      6115

    accuracy                           0.96    174304
   macro avg       0.96      0.91      0.94    174304
weighted avg       0.96      0.96      0.96    174304

[[60323  3106     0     0    14     2    69]
 [ 1934 82855   110     2    62    67    15]
 [    1   142 10445    19     7   133     0]
 [    0     0    89   693     0    22     0]
 [   29   568    29     0  2206    12     0]
 [    2   134   237    21     3  4838     0]
 [  303    29     0     0     0     0  5783]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('Training Accuracy:',abcl_RF.score(x_train,y_train))
print('Testing Accuracy:',abcl_RF.score(x_test,y_test))

Training Accuracy: 0.7354686900675669
Testing Accuracy: 0.7069373049384983


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.57      0.99      0.73     63514
           2       0.96      0.51      0.66     85045
           3       0.89      0.93      0.91     10747
           4       0.90      0.68      0.77       804
           5       0.96      0.47      0.64      2844
           6       0.87      0.78      0.82      5235
           7       1.00      0.16      0.28      6115

    accuracy                           0.71    174304
   macro avg       0.88      0.65      0.69    174304
weighted avg       0.81      0.71      0.69    174304

[[63148   364     0     0     2     0     0]
 [41564 43088   193     2    38   158     2]
 [   42   228 10021    44     7   405     0]
 [    0     0   200   547     0    57     0]
 [  671   788    30     0  1349     6     0]
 [   77   221   834    18     2  4083     0]
 [ 5124     5     0     0     0     0   986]]


### Gradient Boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 0.9988468385180522
Testing Score: 0.9698113640536075


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.98      0.97      0.97     63514
           2       0.97      0.98      0.98     85045
           3       0.95      0.95      0.95     10747
           4       0.74      0.76      0.75       804
           5       0.94      0.86      0.90      2844
           6       0.93      0.91      0.92      5235
           7       0.98      0.96      0.97      6115

    accuracy                           0.97    174304
   macro avg       0.93      0.91      0.92    174304
weighted avg       0.97      0.97      0.97    174304

[[61635  1750     3     5    15     6   100]
 [ 1231 83470   120     8   121    81    14]
 [    4   143 10245   139     8   208     0]
 [    0    16   128   613     0    47     0]
 [   26   347    27     0  2432    12     0]
 [    5   138   248    61     2  4780     1]
 [  215    32     0     1     0     0  5867]]


## Transformations nd Outliers



*   Making three new dataframes forest_sqrt, forest_log, forest_cos
*   Storing the square root transformation of all numerical columns in forest_sqrt
*   Storing the log transformation of all numerical columns in forest_log
*   Storing the cos transformation of all numerical columns in forest_cos



In [ ]:
forest_sqrt = pd.DataFrame()
forest_log = pd.DataFrame()
forest_cos = pd.DataFrame()
for i in quant_col:
  forest_sqrt[i] = np.sqrt(forest_a[i])
  forest_log[i] = np.log(forest_a[i]+1)
  forest_cos[i] = np.cos(forest_a[i])

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Calculating the number of outliers in each numerical columns of original data after feature engineering

In [ ]:
outliers = []
for i in quant_col:
    q1 = forest_a[i].quantile(0.25)
    q3 = forest_a[i].quantile(0.75)
    IQR = q3- q1
    outlier = forest_a[i][(forest_a[i] < (q1 - IQR*1.5)) | (forest_a[i] > (q3 + 1.5*IQR))].count()
    outliers.append(outlier)

Calculating the number of outliers in each numerical columns of square root transformed data

In [ ]:
outliers_sqrt = []
for i in quant_col:
    q1 = forest_sqrt[i].quantile(0.25)
    q3 = forest_sqrt[i].quantile(0.75)
    IQR = q3- q1
    outlier_sqrt = forest_sqrt[i][(forest_sqrt[i] < (q1 - IQR*1.5)) | (forest_sqrt[i] > (q3 + 1.5*IQR))].count()
    outliers_sqrt.append(outlier_sqrt)

Calculating the number of outliers in each numerical columns of log transformed data

In [ ]:
outliers_log = []
for i in quant_col:
    q1 = forest_log[i].quantile(0.25)
    q3 = forest_log[i].quantile(0.75)
    IQR = q3- q1
    outlier_log = forest_log[i][(forest_log[i] < (q1 - IQR*1.5)) | (forest_log[i] > (q3 + 1.5*IQR))].count()
    outliers_log.append(outlier_log)

Calculating the number of outliers in each numerical columns of cos transformed data

In [ ]:
outliers_cos = []
for i in quant_col:
    q1 = forest_cos[i].quantile(0.25)
    q3 = forest_cos[i].quantile(0.75)
    IQR = q3- q1
    outlier_cos = forest_cos[i][(forest_cos[i] < (q1 - IQR*1.5)) | (forest_cos[i] > (q3 + 1.5*IQR))].count()
    outliers_cos.append(outlier_cos)

Constructing a new dataframe and listing the number of outliers after each transformation to their respective columns

In [ ]:
outliers = pd.DataFrame({'columns': quant_col, 'outlier':outliers, 
                          'sqrt_outlier': outliers_sqrt, 
                          'log_outlier': outliers_log,
                          'cos_outlier':outliers_cos})
outliers

,columns,outlier,sqrt_outlier,log_outlier,cos_outlier
0,Elevation,15569,19308,22889,0
1,Aspect,0,0,14762,0
2,Slope,15316,7331,12186,0
3,Horizontal_Distance_To_Hydrology,14557,1262,24603,0
4,Vertical_Distance_To_Hydrology,31463,4380,4858,0
5,Horizontal_Distance_To_Roadways,669,0,10940,0
6,Hillshade_9am,17433,21984,26073,0
7,Hillshade_Noon,15672,18866,21368,0
8,Hillshade_3pm,7832,14563,21476,0
9,Horizontal_Distance_To_Fire_Points,31157,8120,11907,0


*   Will be taking Elevation, Aspect and all Hillshade columns original whereas the degree columns (slope & hydr_slope) to be cos transformed, vertical_scaled log transformed and others square root transformed; so as to have least outliers in each column.
*   We won't be dropping any outliers because they contain major data of least distributed cover_types.



## Transformed Data

In [ ]:
forest_b = pd.DataFrame()
forest_b['Elevation'] = forest_a['Elevation']
forest_b['Aspect'] = forest_a['Aspect']
forest_b['Slope'] = forest_cos['Slope']
forest_b['Horizontal_Distance_To_Hydrology'] = forest_sqrt['Horizontal_Distance_To_Hydrology']
forest_b['Vertical_Scaled'] = forest_log['Vertical_Scaled']
forest_b['Horizontal_Distance_To_Roadways'] = forest_sqrt['Horizontal_Distance_To_Roadways']
forest_b['Hillshade_9am'] = forest_a['Hillshade_9am']
forest_b['Hillshade_Noon'] = forest_a['Hillshade_Noon']
forest_b['Hillshade_3pm'] = forest_a['Hillshade_3pm']
forest_b['Horizontal_Distance_To_Fire_Points'] = forest_sqrt['Horizontal_Distance_To_Fire_Points']
forest_b['Hydr_Slope'] = forest_cos['Hydr_Slope']
forest_b['Avg_Horizontal'] = forest_sqrt['Avg_Horizontal']
forest_b['Fire_Road'] = forest_sqrt['Fire_Road']
forest_b['Hillshades'] = forest_a['Hillshades']
forest_b['Hydr_Dist'] = forest_sqrt['Hydr_Dist']
forest_b['Fire_Hydr'] = forest_sqrt['Fire_Hydr']
forest_b['Wilderness'] = forest_a['Wilderness'].astype('category')
forest_b['Soil'] = forest_a['Soil'].astype('category')
forest_b['Cover_Type'] = forest_a['Cover_Type'].astype('category')

In [ ]:
forest_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 19 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           581012 non-null  int64   
 1   Aspect                              581012 non-null  int64   
 2   Slope                               581012 non-null  float64 
 3   Horizontal_Distance_To_Hydrology    581012 non-null  float64 
 4   Vertical_Scaled                     581012 non-null  float64 
 5   Horizontal_Distance_To_Roadways     581012 non-null  float64 
 6   Hillshade_9am                       581012 non-null  int64   
 7   Hillshade_Noon                      581012 non-null  int64   
 8   Hillshade_3pm                       581012 non-null  int64   
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  float64 
 10  Hydr_Slope                          581012 non-null  float64 
 11  Avg_Horizonta

## Modelling on transformed data

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data into train and test in 7:3 ratio 

In [ ]:
x = forest_b.drop(columns = ['Cover_Type'])
y = forest_b['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9350492245050503
Testing Score: 0.9074834771433816


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.90      0.90      0.90     63514
           2       0.92      0.92      0.92     85045
           3       0.90      0.92      0.91     10747
           4       0.81      0.81      0.81       804
           5       0.79      0.76      0.77      2844
           6       0.85      0.82      0.83      5235
           7       0.95      0.91      0.93      6115

    accuracy                           0.91    174304
   macro avg       0.87      0.86      0.87    174304
weighted avg       0.91      0.91      0.91    174304

[[57240  5907     8     0    74    10   275]
 [ 5557 78425   320     5   471   222    45]
 [    6   290  9854   105    22   470     0]
 [    0     2   117   652     0    33     0]
 [   55   576    42     0  2156    15     0]
 [   14   251   620    46    15  4289     0]
 [  496    56     1     0     0     0  5562]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.423955
1,Aspect,0.013742
2,Slope,0.000631
3,Horizontal_Distance_To_Hydrology,0.014035
4,Vertical_Scaled,0.023873
5,Horizontal_Distance_To_Roadways,0.077315
6,Hillshade_9am,0.024272
7,Hillshade_Noon,0.018327
8,Hillshade_3pm,0.011487
9,Horizontal_Distance_To_Fire_Points,0.055136


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9631652192727952
Testing Score: 0.9416938222874977


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.97      0.95     85045
           3       0.93      0.96      0.94     10747
           4       0.90      0.84      0.87       804
           5       0.93      0.71      0.81      2844
           6       0.93      0.87      0.90      5235
           7       0.97      0.93      0.95      6115

    accuracy                           0.94    174304
   macro avg       0.94      0.89      0.91    174304
weighted avg       0.94      0.94      0.94    174304

[[58832  4537     1     0    17     3   124]
 [ 2516 82085   193     6   124    99    22]
 [    2   199 10286    44     4   212     0]
 [    0     0   112   673     0    19     0]
 [   36   705    55     0  2033    15     0]
 [    0   199   440    28     3  4565     0]
 [  406    42     0     0     0     0  5667]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.278344
1,Aspect,0.022179
2,Slope,0.001400
3,Horizontal_Distance_To_Hydrology,0.022894
4,Vertical_Scaled,0.025380
5,Horizontal_Distance_To_Roadways,0.066141
6,Hillshade_9am,0.022156
7,Hillshade_Noon,0.018386
8,Hillshade_3pm,0.017601
9,Horizontal_Distance_To_Fire_Points,0.050773


### KNN


*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_b.drop(columns= ['Cover_Type','Soil','Wilderness'])
y_ = forest_b['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_b[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Scaled,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Hydr_Slope,Avg_Horizontal,Fire_Road,Hillshades,Hydr_Dist,Fire_Hydr,Soil,Wilderness
0,-1.297805,-0.935157,-1.389008,0.164973,-0.867052,-1.395274,0.330743,0.439143,0.142960,2.587999,1.300680,1.054833,1.068195,0.530152,0.135858,2.574818,29,1
1,-1.319235,-0.890480,-0.578038,-0.056053,-1.014305,-1.567204,0.293388,0.590899,0.221342,2.564187,-0.148505,0.979149,1.008612,0.645367,-0.082435,2.529746,29,1
2,-0.554907,-0.148836,-1.277558,0.210340,0.464875,0.655217,0.816364,0.742654,-0.196691,2.518036,0.587323,1.849922,1.855709,0.668410,0.215057,2.507987,12,1
3,-0.622768,-0.005869,0.943243,0.090516,1.305345,0.606471,0.965786,0.742654,-0.536343,2.557997,0.819799,1.842302,1.855709,0.461022,0.186241,2.536981,30,1
4,-1.301377,-0.988770,-0.578038,-0.378421,-0.891238,-1.565670,0.293388,0.540313,0.195215,2.540716,1.205062,0.940499,0.990654,0.599281,-0.401841,2.478848,29,1


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9225433724986231


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.93      0.92      0.92     63514
           2       0.93      0.94      0.93     85045
           3       0.91      0.91      0.91     10747
           4       0.87      0.82      0.85       804
           5       0.82      0.76      0.79      2844
           6       0.83      0.84      0.84      5235
           7       0.94      0.93      0.94      6115

    accuracy                           0.92    174304
   macro avg       0.89      0.87      0.88    174304
weighted avg       0.92      0.92      0.92    174304

[[58187  4946     7     0    56    14   304]
 [ 4145 79882   319     6   375   271    47]
 [   10   285  9823    66    35   528     0]
 [    0     3    99   663     0    39     0]
 [   83   541    43     0  2156    20     1]
 [   17   248   508    29    23  4410     0]
 [  391    42     0     0     0     0  5682]]


### Ensemble Techniques on transformed data

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Accuracy:',bgcl_DT.score(x_train,y_train))
print('Testing Accuracy:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.9412158108520118
Training Accuracy: 0.9670771167520679
Testing Accuracy: 0.9446140077106664


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.93      0.96      0.95     10747
           4       0.90      0.85      0.88       804
           5       0.91      0.77      0.83      2844
           6       0.93      0.88      0.90      5235
           7       0.97      0.93      0.95      6115

    accuracy                           0.94    174304
   macro avg       0.93      0.90      0.91    174304
weighted avg       0.94      0.94      0.94    174304

[[59298  4031     1     0    31     1   152]
 [ 2700 81869   164     5   162   111    34]
 [    2   181 10291    44     8   221     0]
 [    0     0   101   684     0    19     0]
 [   38   556    49     0  2180    21     0]
 [    4   155   418    23     5  4630     0]
 [  386    31     0     0     0     0  5698]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Accuracy:',bgcl_RF.score(x_train,y_train))
print('Testing Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.928985906350502
Training Accuracy: 0.9482159190377372
Testing Accuracy: 0.9295483752524325


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.91      0.93     63514
           2       0.92      0.96      0.94     85045
           3       0.91      0.95      0.93     10747
           4       0.90      0.80      0.85       804
           5       0.93      0.64      0.76      2844
           6       0.92      0.84      0.87      5235
           7       0.97      0.90      0.94      6115

    accuracy                           0.93    174304
   macro avg       0.93      0.86      0.89    174304
weighted avg       0.93      0.93      0.93    174304

[[57859  5482     1     0    15     8   149]
 [ 2937 81604   226     5   118   129    26]
 [    3   281 10189    40     2   232     0]
 [    0     0   141   646     0    17     0]
 [   40   928    50     0  1815    11     0]
 [    0   276   550    28     1  4380     0]
 [  542    42     0     0     0     0  5531]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Accuracy:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Accuracy:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9215235500654032
Training Accuracy: 0.9998377214119221
Testing Accuracy: 0.9233236185055994


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.93      0.91      0.92     63514
           2       0.93      0.94      0.94     85045
           3       0.91      0.92      0.91     10747
           4       0.87      0.82      0.84       804
           5       0.82      0.75      0.79      2844
           6       0.84      0.84      0.84      5235
           7       0.94      0.93      0.94      6115

    accuracy                           0.92    174304
   macro avg       0.89      0.87      0.88    174304
weighted avg       0.92      0.92      0.92    174304

[[58108  5037     6     0    52    14   297]
 [ 3946 80094   325     6   358   272    44]
 [   10   285  9854    65    33   500     0]
 [    0     3   102   657     0    42     0]
 [   80   556    45     0  2141    21     1]
 [   17   260   509    30    19  4400     0]
 [  391    39     0     0     0     0  5685]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('Training Accuracy:',abcl_DT.score(x_train,y_train))
print('Testing Accuracy:',abcl_DT.score(x_test,y_test))

Training Accuracy: 0.9999237782389331
Testing Accuracy: 0.9446197448136588


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.94      0.96      0.95     10747
           4       0.95      0.81      0.87       804
           5       0.96      0.71      0.82      2844
           6       0.94      0.89      0.91      5235
           7       0.98      0.93      0.95      6115

    accuracy                           0.94    174304
   macro avg       0.95      0.89      0.92    174304
weighted avg       0.94      0.94      0.94    174304

[[59245  4182     0     0     9     0    78]
 [ 2682 82060   158     2    60    69    14]
 [    3   196 10357    13     3   175     0]
 [    0     0   128   650     0    26     0]
 [   42   725    31     0  2032    14     0]
 [    1   194   369    20     2  4649     0]
 [  417    40     0     0     0     0  5658]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('Training Accuracy:',abcl_RF.score(x_train,y_train))
print('Testing Accuracy:',abcl_RF.score(x_test,y_test))

Training Accuracy: 0.6407840514570651
Testing Accuracy: 0.6122292087387553


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.50      1.00      0.66     63514
           2       0.97      0.32      0.48     85045
           3       0.87      0.93      0.90     10747
           4       0.87      0.70      0.77       804
           5       0.96      0.40      0.57      2844
           6       0.88      0.72      0.79      5235
           7       1.00      0.08      0.14      6115

    accuracy                           0.61    174304
   macro avg       0.86      0.59      0.62    174304
weighted avg       0.79      0.61      0.57    174304

[[63455    58     0     0     1     0     0]
 [57318 27291   252     2    42   140     0]
 [  126   230 10007    57     7   320     0]
 [    1     0   196   559     0    48     0]
 [ 1196   469    30     0  1143     6     0]
 [  254   181   993    21     1  3785     0]
 [ 5641     0     0     0     0     0   474]]


### Gradient Boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 0.9987779930564434
Testing Score: 0.9688131081329172


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.98      0.97      0.97     63514
           2       0.97      0.98      0.98     85045
           3       0.95      0.95      0.95     10747
           4       0.75      0.78      0.76       804
           5       0.92      0.84      0.88      2844
           6       0.93      0.90      0.92      5235
           7       0.98      0.96      0.97      6115

    accuracy                           0.97    174304
   macro avg       0.92      0.91      0.92    174304
weighted avg       0.97      0.97      0.97    174304

[[61594  1784     3     6    21     4   102]
 [ 1219 83418   131     9   151    99    18]
 [    4   159 10210   140    26   208     0]
 [    3    11   120   627     1    42     0]
 [   29   374    27     1  2401    12     0]
 [    8   145   289    57     4  4732     0]
 [  197    29     0     1     1     1  5886]]


## Smote

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
x = forest_a.drop(columns = ['Cover_Type'])
y = forest_a['Cover_Type']

Undersampling cover_type '1' & '2' a little as they are most densely distributed cover_types. Also, without loosing much on their given information.

In [ ]:
rus = RandomUnderSampler(sampling_strategy = {1:190000,2:250000}, random_state=25)
X_rus, y_rus = rus.fit_sample(x, y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
a = pd.DataFrame({'y_rus':y_rus})
a['y_rus'].value_counts()

2    250000
1    190000
3     35754
7     20510
6     17367
5      9493
4      2747
Name: y_rus, dtype: int64

Oversampling the other cover_types based on their original distribution, without disturbing data too much.

In [ ]:
smote = SMOTE(sampling_strategy = {3:44000,4:6000,5:15000,6:25000,7:30000}, random_state = 25)

In [ ]:
x_smote, y_smote = smote.fit_sample(X_rus,y_rus)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [ ]:
a = pd.DataFrame({'Cover_Type':y_smote})

In [ ]:
a['Cover_Type'].value_counts()

2    250000
1    190000
3     44000
7     30000
6     25000
5     15000
4      6000
Name: Cover_Type, dtype: int64

In [ ]:
b = pd.DataFrame(x_smote)
b.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,3192.0,80.0,11.0,30.0,7.0,1670.0,234.0,221.0,118.0,1689.0,1.0,29.0,180.0,30.805844,191.000000,3359.0,1129.666667,859.5,13.134022
1,3288.0,39.0,16.0,671.0,212.0,722.0,219.0,204.0,116.0,834.0,1.0,22.0,385.0,703.693825,179.666667,1556.0,742.333333,752.5,17.533741
2,3271.0,274.0,11.0,626.0,37.0,601.0,192.0,243.0,191.0,716.0,3.0,32.0,210.0,627.092497,208.666667,1317.0,647.666667,671.0,3.382556
3,3223.0,75.0,17.0,85.0,9.0,1100.0,238.0,207.0,94.0,942.0,3.0,39.0,182.0,85.475143,179.666667,2042.0,709.000000,513.5,6.044092
4,3243.0,199.0,17.0,150.0,28.0,927.0,212.0,252.0,170.0,1087.0,1.0,23.0,201.0,152.590956,211.333333,2014.0,721.333333,618.5,10.573523


Storing the data after doing smote in a dataframe named 'forest_smote1'

In [ ]:
forest_smote1 = b.copy()
forest_smote1.columns = forest_a.drop(columns = ['Cover_Type']).columns
forest_smote1.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness,Soil,Vertical_Scaled,Hydr_Dist,Hillshades,Fire_Road,Avg_Horizontal,Fire_Hydr,Hydr_Slope
0,3192.0,80.0,11.0,30.0,7.0,1670.0,234.0,221.0,118.0,1689.0,1.0,29.0,180.0,30.805844,191.000000,3359.0,1129.666667,859.5,13.134022
1,3288.0,39.0,16.0,671.0,212.0,722.0,219.0,204.0,116.0,834.0,1.0,22.0,385.0,703.693825,179.666667,1556.0,742.333333,752.5,17.533741
2,3271.0,274.0,11.0,626.0,37.0,601.0,192.0,243.0,191.0,716.0,3.0,32.0,210.0,627.092497,208.666667,1317.0,647.666667,671.0,3.382556
3,3223.0,75.0,17.0,85.0,9.0,1100.0,238.0,207.0,94.0,942.0,3.0,39.0,182.0,85.475143,179.666667,2042.0,709.000000,513.5,6.044092
4,3243.0,199.0,17.0,150.0,28.0,927.0,212.0,252.0,170.0,1087.0,1.0,23.0,201.0,152.590956,211.333333,2014.0,721.333333,618.5,10.573523


In [ ]:
forest_smote1['Cover_Type'] = a['Cover_Type']

In [ ]:
forest_smote1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560000 entries, 0 to 559999
Data columns (total 20 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Elevation                           560000 non-null  float64
 1   Aspect                              560000 non-null  float64
 2   Slope                               560000 non-null  float64
 3   Horizontal_Distance_To_Hydrology    560000 non-null  float64
 4   Vertical_Distance_To_Hydrology      560000 non-null  float64
 5   Horizontal_Distance_To_Roadways     560000 non-null  float64
 6   Hillshade_9am                       560000 non-null  float64
 7   Hillshade_Noon                      560000 non-null  float64
 8   Hillshade_3pm                       560000 non-null  float64
 9   Horizontal_Distance_To_Fire_Points  560000 non-null  float64
 10  Wilderness                          560000 non-null  float64
 11  Soil                      

Converting Wilderness, soil and cover_type columns to category.

In [ ]:
col = ['Wilderness','Soil','Cover_Type']
for i in col:
  forest_smote1[i] = forest_smote1[i].astype('category')

In [ ]:
forest_smote1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560000 entries, 0 to 559999
Data columns (total 20 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           560000 non-null  float64 
 1   Aspect                              560000 non-null  float64 
 2   Slope                               560000 non-null  float64 
 3   Horizontal_Distance_To_Hydrology    560000 non-null  float64 
 4   Vertical_Distance_To_Hydrology      560000 non-null  float64 
 5   Horizontal_Distance_To_Roadways     560000 non-null  float64 
 6   Hillshade_9am                       560000 non-null  float64 
 7   Hillshade_Noon                      560000 non-null  float64 
 8   Hillshade_3pm                       560000 non-null  float64 
 9   Horizontal_Distance_To_Fire_Points  560000 non-null  float64 
 10  Wilderness                          560000 non-null  category
 11  Soil         

## Modelling on Smote data

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data into train and test in 7:3 ratio

In [ ]:
x = forest_smote1.drop(columns = ['Cover_Type','Vertical_Scaled'])
y = forest_smote1['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9347780612244898
Testing Score: 0.9044642857142857


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.90      0.89      0.89     57141
           2       0.91      0.91      0.91     75000
           3       0.92      0.92      0.92     13131
           4       0.91      0.88      0.89      1797
           5       0.85      0.83      0.84      4468
           6       0.87      0.87      0.87      7534
           7       0.95      0.95      0.95      8929

    accuracy                           0.90    168000
   macro avg       0.90      0.89      0.90    168000
weighted avg       0.90      0.90      0.90    168000

[[50959  5699     6     0    93    24   360]
 [ 5351 68574   276     1   497   254    47]
 [    1   293 12079   125    30   603     0]
 [    0     4   157  1582     0    54     0]
 [   66   634    38     0  3705    25     0]
 [   18   281   601    39    18  6577     0]
 [  375    80     0     0     0     0  8474]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.440129
1,Aspect,0.014991
2,Slope,0.006444
3,Horizontal_Distance_To_Hydrology,0.017903
4,Vertical_Distance_To_Hydrology,0.015585
5,Horizontal_Distance_To_Roadways,0.067705
6,Hillshade_9am,0.023299
7,Hillshade_Noon,0.015257
8,Hillshade_3pm,0.011112
9,Horizontal_Distance_To_Fire_Points,0.052241


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train,y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9679821428571429
Testing Score: 0.9446130952380952


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.96      0.92      0.94     57141
           2       0.94      0.96      0.95     75000
           3       0.94      0.96      0.95     13131
           4       0.94      0.92      0.93      1797
           5       0.95      0.87      0.91      4468
           6       0.93      0.92      0.93      7534
           7       0.97      0.97      0.97      8929

    accuracy                           0.94    168000
   macro avg       0.95      0.93      0.94    168000
weighted avg       0.94      0.94      0.94    168000

[[52726  4175     2     0    28     9   201]
 [ 2184 72236   219     2   170   152    37]
 [    0   134 12628    62     9   298     0]
 [    0     0   112  1662     0    23     0]
 [   16   529    36     0  3873    14     0]
 [    0   142   407    41     3  6941     0]
 [  263    34     0     0     3     0  8629]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.276939
1,Aspect,0.021180
2,Slope,0.012026
3,Horizontal_Distance_To_Hydrology,0.022812
4,Vertical_Distance_To_Hydrology,0.019907
5,Horizontal_Distance_To_Roadways,0.064350
6,Hillshade_9am,0.021471
7,Hillshade_Noon,0.016686
8,Hillshade_3pm,0.015512
9,Horizontal_Distance_To_Fire_Points,0.046639


### KNN


*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_smote1.drop(columns= ['Cover_Type','Soil','Wilderness','Vertical_Scaled'])
y_ = forest_smote1['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_smote1[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Hydr_Dist,Hillshades,Fire_Road,Avg_Horizontal,Fire_Hydr,Hydr_Slope,Soil,Wilderness
0,0.824993,-0.680594,-0.444941,-1.109565,-0.684543,-0.397925,0.806948,-0.088893,-0.613490,-0.184734,-1.114369,-0.083351,-0.364115,-0.458914,-0.357311,0.430281,29.0,1.0
1,1.139856,-1.046790,0.209780,1.895358,2.796892,-1.010570,0.259169,-0.929194,-0.664590,-0.838546,1.973946,-0.846651,-1.129868,-0.946347,-0.517189,0.925406,22.0,1.0
2,1.084099,1.052138,-0.444941,1.684404,-0.175065,-1.088767,-0.726835,0.998555,1.251671,-0.928780,1.622374,1.106501,-1.231374,-1.065479,-0.638965,-0.667104,32.0,3.0
3,0.926668,-0.725252,0.340724,-0.851732,-0.650578,-0.766288,0.953023,-0.780906,-1.226693,-0.755959,-0.863457,-0.846651,-0.923459,-0.988295,-0.874300,-0.367587,39.0,3.0
4,0.992264,0.382268,0.340724,-0.547021,-0.327908,-0.878089,0.003538,1.443421,0.715118,-0.645079,-0.555419,1.286101,-0.935351,-0.972774,-0.717410,0.142135,23.0,1.0


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9474285714285714


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.95      0.94      0.95     57141
           2       0.95      0.95      0.95     75000
           3       0.95      0.95      0.95     13131
           4       0.92      0.93      0.93      1797
           5       0.89      0.93      0.91      4468
           6       0.90      0.93      0.91      7534
           7       0.96      0.98      0.97      8929

    accuracy                           0.95    168000
   macro avg       0.93      0.94      0.94    168000
weighted avg       0.95      0.95      0.95    168000

[[53737  3011     2     0    72    14   305]
 [ 2626 71438   205     6   395   284    46]
 [    6   131 12440    94    22   438     0]
 [    0     5    77  1679     0    36     0]
 [   36   231    25     0  4166     9     1]
 [   10   157   338    44    11  6974     0]
 [  173    21     0     0     1     0  8734]]


### Ensemble Tecniques on Smote data

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Accuracy:',bgcl_DT.score(x_train,y_train))
print('Testing Accuracy:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.9428928571428571
Training Accuracy: 0.9686377551020409
Testing Accuracy: 0.9446964285714285


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     57141
           2       0.94      0.96      0.95     75000
           3       0.95      0.96      0.96     13131
           4       0.94      0.92      0.93      1797
           5       0.93      0.87      0.90      4468
           6       0.94      0.93      0.93      7534
           7       0.97      0.96      0.97      8929

    accuracy                           0.94    168000
   macro avg       0.95      0.93      0.94    168000
weighted avg       0.94      0.94      0.94    168000

[[53172  3688     1     0    49     9   222]
 [ 2667 71763   165     1   226   138    40]
 [    0   142 12644    65    13   267     0]
 [    0     2   114  1650     0    31     0]
 [   30   478    34     0  3908    18     0]
 [    3   133   366    34     7  6991     0]
 [  306    40     0     0     2     0  8581]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Accuracy:',bgcl_RF.score(x_train,y_train))
print('Testing Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9328137755102041
Training Accuracy: 0.9519668367346938
Testing Accuracy: 0.9318511904761905


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.91      0.92     57141
           2       0.92      0.95      0.94     75000
           3       0.93      0.96      0.94     13131
           4       0.93      0.91      0.92      1797
           5       0.95      0.81      0.87      4468
           6       0.92      0.90      0.91      7534
           7       0.97      0.95      0.96      8929

    accuracy                           0.93    168000
   macro avg       0.94      0.91      0.92    168000
weighted avg       0.93      0.93      0.93    168000

[[51820  5032     1     0    33    16   239]
 [ 2699 71622   288     3   170   181    37]
 [    1   183 12546    69     4   328     0]
 [    0     0   134  1639     0    24     0]
 [   22   745    53     0  3633    15     0]
 [    0   202   516    43     2  6771     0]
 [  368    39     0     0     2     0  8520]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Accuracy:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Accuracy:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9462882653061224
Training Accuracy: 0.999859693877551
Testing Accuracy: 0.9465357142857143


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.95      0.94      0.94     57141
           2       0.95      0.95      0.95     75000
           3       0.95      0.95      0.95     13131
           4       0.92      0.93      0.93      1797
           5       0.89      0.93      0.91      4468
           6       0.89      0.92      0.91      7534
           7       0.96      0.98      0.97      8929

    accuracy                           0.95    168000
   macro avg       0.93      0.94      0.94    168000
weighted avg       0.95      0.95      0.95    168000

[[53660  3057     4     0    77    15   328]
 [ 2588 71426   222     5   416   293    50]
 [    5   127 12421    97    21   460     0]
 [    0     5    78  1678     0    36     0]
 [   35   245    25     0  4149    13     1]
 [    9   154   363    47    12  6949     0]
 [  167    26     0     0     1     0  8735]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('Training Accuracy:',abcl_DT.score(x_train,y_train))
print('Testing Accuracy:',abcl_DT.score(x_test,y_test))

Training Accuracy: 0.9999846938775511
Testing Accuracy: 0.9593571428571429


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.96      0.95      0.95     57141
           2       0.95      0.97      0.96     75000
           3       0.97      0.98      0.97     13131
           4       0.97      0.94      0.96      1797
           5       0.97      0.90      0.94      4468
           6       0.97      0.96      0.96      7534
           7       0.98      0.97      0.98      8929

    accuracy                           0.96    168000
   macro avg       0.97      0.95      0.96    168000
weighted avg       0.96      0.96      0.96    168000

[[54026  2986     0     0    15     1   113]
 [ 2010 72670   123     0    87    87    23]
 [    0   102 12845    34     4   146     0]
 [    0     0    77  1698     0    22     0]
 [   21   391    20     0  4031     5     0]
 [    2   111   199    19     1  7202     0]
 [  208    19     0     0     2     0  8700]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('Training Accuracy:',abcl_RF.score(x_train,y_train))
print('Testing Accuracy:',abcl_RF.score(x_test,y_test))

Training Accuracy: 0.8682015306122449
Testing Accuracy: 0.8300059523809524


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.87      0.71      0.78     57141
           2       0.88      0.89      0.88     75000
           3       0.91      0.91      0.91     13131
           4       0.93      0.82      0.87      1797
           5       0.87      0.83      0.85      4468
           6       0.82      0.87      0.85      7534
           7       0.48      0.98      0.64      8929

    accuracy                           0.83    168000
   macro avg       0.82      0.86      0.82    168000
weighted avg       0.85      0.83      0.83    168000

[[40303  8560     0     0   163    11  8104]
 [ 6000 66750   181     1   372   350  1346]
 [   12   156 11927    83    21   932     0]
 [    0     0   228  1469     0   100     0]
 [   61   666    28     0  3689    24     0]
 [    7   146   785    24     8  6564     0]
 [  181     6     0     0     3     0  8739]]


### Gradient Boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 1.0
Testing Score: 0.9734404761904762


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.98      0.97      0.97     57141
           2       0.97      0.98      0.98     75000
           3       0.97      0.98      0.97     13131
           4       0.96      0.95      0.95      1797
           5       0.96      0.93      0.94      4468
           6       0.96      0.96      0.96      7534
           7       0.98      0.98      0.98      8929

    accuracy                           0.97    168000
   macro avg       0.97      0.96      0.97    168000
weighted avg       0.97      0.97      0.97    168000

[[55413  1559     0     0    20     8   141]
 [ 1214 73433   101     0   154    77    21]
 [    0   101 12813    58    10   149     0]
 [    0     0    68  1703     0    26     0]
 [   17   259    18     0  4170     4     0]
 [    0    88   182    21     2  7241     0]
 [  142    20     0     0     2     0  8765]]


## Smote_Outliers & Transformation



*   Making three new dataframes forest_sqrt, forest_log, forest_cos
*   Storing the square root transformation of all numerical columns of smote data in forest_sqrt
*   Storing the log transformation of all numerical columns of smote data in forest_log
*   Storing the cos transformation of all numerical columns of smote data in forest_cos



In [ ]:
forest_sqrt = pd.DataFrame()
forest_log = pd.DataFrame()
forest_cos = pd.DataFrame()
for i in quant_col:
  forest_sqrt[i] = np.sqrt(forest_smote1[i])
  forest_log[i] = np.log(forest_smote1[i]+1)
  forest_cos[i] = np.cos(forest_smote1[i])

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Calculating the number of outliers in each numerical columns of smote data

In [ ]:
outliers = []
for i in quant_col:
    q1 = forest_smote1[i].quantile(0.25)
    q3 = forest_smote1[i].quantile(0.75)
    IQR = q3- q1
    outlier = forest_smote1[i][(forest_smote1[i] < (q1 - IQR*1.5)) | (forest_smote1[i] > (q3 + 1.5*IQR))].count()
    outliers.append(outlier)

Calculating the number of outliers in each numerical columns of square root transformed data

In [ ]:
outliers_sqrt = []
for i in quant_col:
    q1 = forest_sqrt[i].quantile(0.25)
    q3 = forest_sqrt[i].quantile(0.75)
    IQR = q3- q1
    outlier_sqrt = forest_sqrt[i][(forest_sqrt[i] < (q1 - IQR*1.5)) | (forest_sqrt[i] > (q3 + 1.5*IQR))].count()
    outliers_sqrt.append(outlier_sqrt)

Calculating the number of outliers in each numerical columns of log transformed data

In [ ]:
outliers_log = []
for i in quant_col:
    q1 = forest_log[i].quantile(0.25)
    q3 = forest_log[i].quantile(0.75)
    IQR = q3- q1
    outlier_log = forest_log[i][(forest_log[i] < (q1 - IQR*1.5)) | (forest_log[i] > (q3 + 1.5*IQR))].count()
    outliers_log.append(outlier_log)

Calculating the number of outliers in each numerical columns of cos transformed data

In [ ]:
outliers_cos = []
for i in quant_col:
    q1 = forest_cos[i].quantile(0.25)
    q3 = forest_cos[i].quantile(0.75)
    IQR = q3- q1
    outlier_cos = forest_cos[i][(forest_cos[i] < (q1 - IQR*1.5)) | (forest_cos[i] > (q3 + 1.5*IQR))].count()
    outliers_cos.append(outlier_cos)

Constructing a new dataframe and listing the number of outliers after each transformation to their respective columns

In [ ]:
outliers = pd.DataFrame({'columns': quant_col, 'outlier':outliers, 
                          'sqrt_outlier': outliers_sqrt, 
                          'log_outlier': outliers_log,
                          'cos_outlier':outliers_cos})
outliers

,columns,outlier,sqrt_outlier,log_outlier,cos_outlier
0,Elevation,12854,16826,20499,0
1,Aspect,0,0,13930,0
2,Slope,8036,1798,11013,0
3,Horizontal_Distance_To_Hydrology,13095,722,25178,0
4,Vertical_Distance_To_Hydrology,30494,4135,4775,0
5,Horizontal_Distance_To_Roadways,1081,0,10606,0
6,Hillshade_9am,16738,22025,26079,0
7,Hillshade_Noon,15104,18137,20472,0
8,Hillshade_3pm,7512,14717,22264,0
9,Horizontal_Distance_To_Fire_Points,28759,7538,10207,0


*   Will be taking Elevation, Aspect and all Hillshade columns original whereas the degree columns (slope & hydr_slope) to be cos transformed, vertical_scaled log transformed and others square root transformed; so as to have least outliers in each column.
*   We won't be dropping any outliers because they contain major data of least distributed cover_types.



## Smote_Transformed

In [ ]:
forest_c = pd.DataFrame()
forest_c['Elevation'] = forest_smote1['Elevation']
forest_c['Aspect'] = forest_smote1['Aspect']
forest_c['Slope'] = forest_cos['Slope']
forest_c['Horizontal_Distance_To_Hydrology'] = forest_sqrt['Horizontal_Distance_To_Hydrology']
forest_c['Vertical_Scaled'] = forest_log['Vertical_Scaled']
forest_c['Horizontal_Distance_To_Roadways'] = forest_sqrt['Horizontal_Distance_To_Roadways']
forest_c['Hillshade_9am'] = forest_smote1['Hillshade_9am']
forest_c['Hillshade_Noon'] = forest_smote1['Hillshade_Noon']
forest_c['Hillshade_3pm'] = forest_smote1['Hillshade_3pm']
forest_c['Horizontal_Distance_To_Fire_Points'] = forest_sqrt['Horizontal_Distance_To_Fire_Points']
forest_c['Hydr_Slope'] = forest_cos['Hydr_Slope']
forest_c['Fire_Hydr'] = forest_sqrt['Fire_Hydr']
forest_c['Avg_Horizontal'] = forest_sqrt['Avg_Horizontal']
forest_c['Fire_Road'] = forest_sqrt['Fire_Road']
forest_c['Hillshades'] = forest_smote1['Hillshades']
forest_c['Hydr_Dist'] = forest_sqrt['Hydr_Dist']
forest_c['Wilderness'] = forest_smote1['Wilderness'].astype('category')
forest_c['Soil'] = forest_smote1['Soil'].astype('category')
forest_c['Cover_Type'] = forest_smote1['Cover_Type'].astype('category')

In [ ]:
forest_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560000 entries, 0 to 559999
Data columns (total 19 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           560000 non-null  float64 
 1   Aspect                              560000 non-null  float64 
 2   Slope                               560000 non-null  float64 
 3   Horizontal_Distance_To_Hydrology    560000 non-null  float64 
 4   Vertical_Scaled                     560000 non-null  float64 
 5   Horizontal_Distance_To_Roadways     560000 non-null  float64 
 6   Hillshade_9am                       560000 non-null  float64 
 7   Hillshade_Noon                      560000 non-null  float64 
 8   Hillshade_3pm                       560000 non-null  float64 
 9   Horizontal_Distance_To_Fire_Points  560000 non-null  float64 
 10  Hydr_Slope                          560000 non-null  float64 
 11  Fire_Hydr    

## Modelling on Smote transformed data

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data into train and test in 7:3 ratio

In [ ]:
x = forest_c.drop(columns = ['Cover_Type'])
y = forest_c['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.933579081632653
Testing Score: 0.9042738095238095


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.90      0.89      0.89     57141
           2       0.91      0.91      0.91     75000
           3       0.92      0.92      0.92     13131
           4       0.91      0.88      0.90      1797
           5       0.85      0.83      0.84      4468
           6       0.87      0.87      0.87      7534
           7       0.96      0.95      0.95      8929

    accuracy                           0.90    168000
   macro avg       0.90      0.89      0.90    168000
weighted avg       0.90      0.90      0.90    168000

[[51035  5637     5     0    93    23   348]
 [ 5418 68508   271     1   503   258    41]
 [    1   270 12065   114    26   655     0]
 [    0     4   151  1590     0    52     0]
 [   80   621    41     0  3703    23     0]
 [   19   267   628    42    24  6554     0]
 [  394    72     0     0     0     0  8463]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.441604
1,Aspect,0.015660
2,Slope,0.000636
3,Horizontal_Distance_To_Hydrology,0.019125
4,Vertical_Scaled,0.022534
5,Horizontal_Distance_To_Roadways,0.069493
6,Hillshade_9am,0.025390
7,Hillshade_Noon,0.016087
8,Hillshade_3pm,0.012656
9,Horizontal_Distance_To_Fire_Points,0.053154


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9660892857142858
Testing Score: 0.9425238095238095


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.95      0.92      0.94     57141
           2       0.93      0.96      0.95     75000
           3       0.94      0.96      0.95     13131
           4       0.94      0.92      0.93      1797
           5       0.95      0.86      0.90      4468
           6       0.93      0.92      0.92      7534
           7       0.97      0.96      0.97      8929

    accuracy                           0.94    168000
   macro avg       0.95      0.93      0.94    168000
weighted avg       0.94      0.94      0.94    168000

[[52615  4270     2     0    35    14   205]
 [ 2305 72101   229     2   175   153    35]
 [    0   141 12617    64     6   303     0]
 [    0     0   108  1661     0    28     0]
 [   18   562    37     0  3837    14     0]
 [    1   139   431    38     2  6923     0]
 [  304    32     0     0     3     0  8590]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.274747
1,Aspect,0.021580
2,Slope,0.001343
3,Horizontal_Distance_To_Hydrology,0.023040
4,Vertical_Scaled,0.023904
5,Horizontal_Distance_To_Roadways,0.063958
6,Hillshade_9am,0.023120
7,Hillshade_Noon,0.018676
8,Hillshade_3pm,0.017703
9,Horizontal_Distance_To_Fire_Points,0.048153


### KNN

*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio


In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_c.drop(columns= ['Cover_Type','Soil','Wilderness'])
y_ = forest_c['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_c[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Scaled,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Hydr_Slope,Fire_Hydr,Avg_Horizontal,Fire_Road,Hillshades,Hydr_Dist,Soil,Wilderness
0,0.824993,-0.680594,0.015748,-1.367684,-0.711715,-0.242203,0.806948,-0.088893,-0.613490,-0.031070,1.082570,-0.242679,-0.360650,-0.245514,-0.083351,-1.371248,29.0,1.0
1,1.139856,-1.046790,-1.343958,1.619367,2.444306,-1.089579,0.259169,-0.929194,-0.664590,-0.884791,0.267644,-0.436214,-0.984879,-1.272567,-0.846651,1.663775,22.0,1.0
2,1.084099,1.052138,0.015748,1.490143,-0.072622,-1.232150,-0.726835,0.998555,1.251671,-1.032981,-1.419384,-0.593060,-1.161074,-1.447657,1.106501,1.448859,32.0,3.0
3,0.926668,-0.725252,-0.379393,-0.820428,-0.665920,-0.708370,0.953023,-0.780906,-1.226693,-0.758116,1.259621,-0.925946,-1.045563,-0.953958,-0.846651,-0.836644,39.0,3.0
4,0.992264,0.382268,-0.379393,-0.377644,-0.254274,-0.873029,0.003538,1.443421,0.715118,-0.598972,-0.645086,-0.699194,-1.022947,-0.971208,1.286101,-0.387040,23.0,1.0


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.92225


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.92      0.91      0.92     57141
           2       0.93      0.93      0.93     75000
           3       0.93      0.93      0.93     13131
           4       0.92      0.91      0.91      1797
           5       0.85      0.88      0.86      4468
           6       0.87      0.90      0.89      7534
           7       0.95      0.97      0.96      8929

    accuracy                           0.92    168000
   macro avg       0.91      0.92      0.91    168000
weighted avg       0.92      0.92      0.92    168000

[[51935  4662     9     0    86    18   431]
 [ 3873 69810   331     8   582   338    58]
 [    6   199 12238    98    29   561     0]
 [    0     4   100  1641     1    51     0]
 [   78   415    41     0  3920    14     0]
 [   19   219   469    46    16  6765     0]
 [  263    37     0     0     0     0  8629]]


### Ensemble Techniques on Smote Transformed data

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Score:',bgcl_DT.score(x_train,y_train))
print('Testing Score:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.942125
Training Score: 0.968
Testing Score: 0.9435714285714286


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     57141
           2       0.94      0.96      0.95     75000
           3       0.95      0.96      0.95     13131
           4       0.94      0.92      0.93      1797
           5       0.93      0.87      0.90      4468
           6       0.93      0.93      0.93      7534
           7       0.97      0.96      0.96      8929

    accuracy                           0.94    168000
   macro avg       0.94      0.93      0.94    168000
weighted avg       0.94      0.94      0.94    168000

[[53066  3787     2     0    45     8   233]
 [ 2689 71731   167     1   236   135    41]
 [    0   149 12605    68    12   297     0]
 [    0     2   117  1647     0    31     0]
 [   29   491    29     0  3902    17     0]
 [    4   135   365    33     8  6989     0]
 [  308    39     0     0     2     0  8580]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Score:',bgcl_RF.score(x_train,y_train))
print('Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9314260204081632
Training Score: 0.9508010204081633
Accuracy: 0.9302738095238096


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.91      0.92     57141
           2       0.92      0.95      0.94     75000
           3       0.92      0.95      0.94     13131
           4       0.93      0.91      0.92      1797
           5       0.94      0.81      0.87      4468
           6       0.92      0.89      0.91      7534
           7       0.97      0.95      0.96      8929

    accuracy                           0.93    168000
   macro avg       0.94      0.91      0.92    168000
weighted avg       0.93      0.93      0.93    168000

[[51730  5112     2     0    34    16   247]
 [ 2741 71539   302     3   180   199    36]
 [    0   194 12526    71     4   336     0]
 [    0     0   132  1638     0    27     0]
 [   21   759    62     0  3610    16     0]
 [    0   201   556    41     2  6734     0]
 [  377    41     0     0     2     0  8509]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Score:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Score:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9216607142857143
Training Score: 0.9997882653061224
Testing Score: 0.9227202380952381


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('AdaBoost DT Training Score:',abcl_DT.score(x_train,y_train))
print('AdaBoost DT Testing Score:',abcl_DT.score(x_test,y_test))

AdaBoost DT Training Score: 0.9999642857142857
AdaBoost DT Testing Score: 0.9449821428571429


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     57141
           2       0.94      0.96      0.95     75000
           3       0.95      0.97      0.96     13131
           4       0.96      0.93      0.95      1797
           5       0.97      0.85      0.91      4468
           6       0.96      0.93      0.94      7534
           7       0.98      0.96      0.97      8929

    accuracy                           0.94    168000
   macro avg       0.96      0.93      0.95    168000
weighted avg       0.95      0.94      0.94    168000

[[52995  4014     1     0     8     2   121]
 [ 2709 71892   167     1    98   114    19]
 [    0   137 12770    38     4   182     0]
 [    0     0   105  1667     0    25     0]
 [   25   587    32     0  3820     4     0]
 [    1   153   320    22     2  7036     0]
 [  320    30     0     0     2     0  8577]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('AdaBoost RF Training Score:',abcl_RF.score(x_train,y_train))
print('AdaBoost RF Testing Score:',abcl_RF.score(x_test,y_test))

AdaBoost RF Training Score: 0.8300484693877551
AdaBoost RF Testing Score: 0.7931785714285714


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.66      0.97      0.78     57141
           2       0.96      0.62      0.75     75000
           3       0.90      0.93      0.92     13131
           4       0.92      0.87      0.89      1797
           5       0.91      0.76      0.83      4468
           6       0.87      0.85      0.86      7534
           7       0.88      0.88      0.88      8929

    accuracy                           0.79    168000
   macro avg       0.87      0.84      0.84    168000
weighted avg       0.84      0.79      0.79    168000

[[55169  1100     0     0    24     6   842]
 [27380 46609   260     1   258   302   190]
 [   15   164 12246   113    25   568     0]
 [    0     0   160  1558     0    79     0]
 [  271   749    36     0  3390    22     0]
 [   36   148   886    28     9  6427     0]
 [ 1066     4     0     0     4     0  7855]]


### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print(gbcl.score(x_train,y_train))
print(gbcl.score(x_test,y_test))

1.0
0.97275


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.98      0.97      0.97     57141
           2       0.97      0.98      0.98     75000
           3       0.97      0.98      0.97     13131
           4       0.95      0.94      0.94      1797
           5       0.96      0.93      0.95      4468
           6       0.96      0.96      0.96      7534
           7       0.98      0.98      0.98      8929

    accuracy                           0.97    168000
   macro avg       0.97      0.96      0.96    168000
weighted avg       0.97      0.97      0.97    168000

[[55303  1668     0     0    23     6   141]
 [ 1193 73476   103     1   135    74    18]
 [    0   104 12806    62     4   155     0]
 [    0     0    79  1688     0    30     0]
 [   19   249    20     0  4172     8     0]
 [    2    86   194    27     2  7223     0]
 [  154    19     0     0     2     0  8754]]


## Smote_Transformed_Original

Constructing a dataframe with original features after applying smote and transforming it.

In [ ]:
forest_e = forest_c.drop(columns = forest_c.columns[10:16])
forest_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560000 entries, 0 to 559999
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           560000 non-null  float64 
 1   Aspect                              560000 non-null  float64 
 2   Slope                               560000 non-null  float64 
 3   Horizontal_Distance_To_Hydrology    560000 non-null  float64 
 4   Vertical_Scaled                     560000 non-null  float64 
 5   Horizontal_Distance_To_Roadways     560000 non-null  float64 
 6   Hillshade_9am                       560000 non-null  float64 
 7   Hillshade_Noon                      560000 non-null  float64 
 8   Hillshade_3pm                       560000 non-null  float64 
 9   Horizontal_Distance_To_Fire_Points  560000 non-null  float64 
 10  Wilderness                          560000 non-null  category
 11  Soil         

## Modelling on smote transformed original data


### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data into train and test in 7:3 ratio

In [ ]:
x = forest_e.drop(columns = ['Cover_Type'])
y = forest_e['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9298903061224489
Testing Score: 0.9005833333333333


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.89      0.89      0.89     57141
           2       0.91      0.91      0.91     75000
           3       0.92      0.91      0.91     13131
           4       0.91      0.89      0.90      1797
           5       0.84      0.84      0.84      4468
           6       0.86      0.87      0.87      7534
           7       0.94      0.94      0.94      8929

    accuracy                           0.90    168000
   macro avg       0.90      0.89      0.89    168000
weighted avg       0.90      0.90      0.90    168000

[[50771  5846     5     0    84    11   424]
 [ 5533 68333   257     1   588   216    72]
 [    8   296 11968   105    30   724     0]
 [    0     6   146  1598     0    47     0]
 [   65   593    52     0  3738    20     0]
 [   34   297   618    52    13  6520     0]
 [  463    94     0     0     2     0  8370]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.448585
1,Aspect,0.020652
2,Slope,0.001047
3,Horizontal_Distance_To_Hydrology,0.046650
4,Vertical_Scaled,0.030602
5,Horizontal_Distance_To_Roadways,0.109451
6,Hillshade_9am,0.025406
7,Hillshade_Noon,0.023538
8,Hillshade_3pm,0.015247
9,Horizontal_Distance_To_Fire_Points,0.116505


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9545841836734694
Testing Score: 0.9291130952380953


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.94      0.90      0.92     57141
           2       0.92      0.95      0.93     75000
           3       0.93      0.95      0.94     13131
           4       0.94      0.93      0.93      1797
           5       0.94      0.82      0.88      4468
           6       0.92      0.90      0.91      7534
           7       0.97      0.95      0.96      8929

    accuracy                           0.93    168000
   macro avg       0.94      0.91      0.92    168000
weighted avg       0.93      0.93      0.93    168000

[[51517  5353     2     0    24    20   225]
 [ 2848 71504   250     1   179   184    34]
 [    4   186 12522    68     8   343     0]
 [    0     0   110  1665     0    22     0]
 [   24   727    51     0  3648    18     0]
 [    2   181   552    38     2  6759     0]
 [  415    36     0     0     2     0  8476]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.310944
1,Aspect,0.029215
2,Slope,0.003131
3,Horizontal_Distance_To_Hydrology,0.044182
4,Vertical_Scaled,0.035856
5,Horizontal_Distance_To_Roadways,0.105092
6,Hillshade_9am,0.028668
7,Hillshade_Noon,0.028786
8,Hillshade_3pm,0.025045
9,Horizontal_Distance_To_Fire_Points,0.093682


### KNN

*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_e.drop(columns= ['Cover_Type','Soil','Wilderness'])
y_ = forest_e['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_e[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Scaled,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Soil,Wilderness
0,0.824993,-0.680594,0.015748,-1.367684,-0.711715,-0.242203,0.806948,-0.088893,-0.613490,-0.031070,29.0,1.0
1,1.139856,-1.046790,-1.343958,1.619367,2.444306,-1.089579,0.259169,-0.929194,-0.664590,-0.884791,22.0,1.0
2,1.084099,1.052138,0.015748,1.490143,-0.072622,-1.232150,-0.726835,0.998555,1.251671,-1.032981,32.0,3.0
3,0.926668,-0.725252,-0.379393,-0.820428,-0.665920,-0.708370,0.953023,-0.780906,-1.226693,-0.758116,39.0,3.0
4,0.992264,0.382268,-0.379393,-0.377644,-0.254274,-0.873029,0.003538,1.443421,0.715118,-0.598972,23.0,1.0


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9183690476190476


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.92      0.90      0.91     57141
           2       0.92      0.93      0.93     75000
           3       0.93      0.93      0.93     13131
           4       0.91      0.92      0.91      1797
           5       0.86      0.87      0.87      4468
           6       0.87      0.89      0.88      7534
           7       0.94      0.96      0.95      8929

    accuracy                           0.92    168000
   macro avg       0.91      0.91      0.91    168000
weighted avg       0.92      0.92      0.92    168000

[[51659  4922     5     0    84    25   446]
 [ 4135 69630   310     5   520   332    68]
 [    5   218 12179   107    31   590     1]
 [    0     1    98  1645     0    53     0]
 [   81   426    39     0  3903    19     0]
 [   19   237   526    49    12  6691     0]
 [  304    46     0     0     0     0  8579]]


### Ensemble on smote transformed original data

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Accuracy:',bgcl_DT.score(x_train,y_train))
print('Testing Accuracy:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.9377474489795918
Training Accuracy: 0.9637704081632653
Testing Accuracy: 0.9398630952380952


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.92      0.93     57141
           2       0.94      0.95      0.94     75000
           3       0.95      0.96      0.95     13131
           4       0.94      0.91      0.93      1797
           5       0.92      0.87      0.89      4468
           6       0.93      0.92      0.93      7534
           7       0.97      0.96      0.96      8929

    accuracy                           0.94    168000
   macro avg       0.94      0.93      0.93    168000
weighted avg       0.94      0.94      0.94    168000

[[52728  4119     3     0    42    10   239]
 [ 2798 71580   180     2   272   127    41]
 [    0   165 12572    74    18   302     0]
 [    0     2   126  1642     0    27     0]
 [   45   498    35     0  3870    20     0]
 [    4   148   384    34     5  6959     0]
 [  350    32     0     0     1     0  8546]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Accuracy:',bgcl_RF.score(x_train,y_train))
print('Testing Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9156454081632653
Training Accuracy: 0.9361173469387755
Testing Accuracy: 0.9147738095238095


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.93      0.88      0.91     57141
           2       0.90      0.94      0.92     75000
           3       0.91      0.94      0.93     13131
           4       0.93      0.90      0.92      1797
           5       0.94      0.75      0.84      4468
           6       0.91      0.87      0.89      7534
           7       0.96      0.93      0.95      8929

    accuracy                           0.91    168000
   macro avg       0.93      0.89      0.91    168000
weighted avg       0.92      0.91      0.91    168000

[[50547  6255     4     0    24    21   290]
 [ 3374 70865   330     1   180   212    38]
 [    2   253 12401    79     2   394     0]
 [    0     0   158  1623     0    16     0]
 [   21   998    64     0  3368    17     0]
 [    9   268   673    43     2  6539     0]
 [  548    40     0     0     2     0  8339]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Accuracy:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Accuracy:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9172729591836735
Training Accuracy: 0.9997423469387755
Testing Accuracy: 0.9179821428571429


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.92      0.90      0.91     57141
           2       0.92      0.93      0.93     75000
           3       0.92      0.93      0.93     13131
           4       0.91      0.91      0.91      1797
           5       0.86      0.86      0.86      4468
           6       0.87      0.89      0.88      7534
           7       0.94      0.96      0.95      8929

    accuracy                           0.92    168000
   macro avg       0.91      0.91      0.91    168000
weighted avg       0.92      0.92      0.92    168000

[[51509  5046    10     0    83    26   467]
 [ 3969 69799   305     4   524   330    69]
 [    4   209 12174   105    28   611     0]
 [    0     1   103  1639     0    54     0]
 [   76   469    46     0  3857    20     0]
 [   17   242   541    45    12  6677     0]
 [  312    50     0     0     1     0  8566]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('Training Accuracy:',abcl_DT.score(x_train,y_train))
print('Testing Accuracy:',abcl_DT.score(x_test,y_test))

Training Accuracy: 0.9999234693877551
Testing Accuracy: 0.9455833333333333


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     57141
           2       0.94      0.96      0.95     75000
           3       0.96      0.97      0.96     13131
           4       0.96      0.93      0.95      1797
           5       0.97      0.86      0.91      4468
           6       0.96      0.94      0.95      7534
           7       0.98      0.96      0.97      8929

    accuracy                           0.95    168000
   macro avg       0.96      0.94      0.95    168000
weighted avg       0.95      0.95      0.95    168000

[[52980  4016     0     0    12     4   129]
 [ 2642 71968   171     0    93   108    18]
 [    0   147 12749    43     5   187     0]
 [    0     0    90  1680     0    27     0]
 [   26   565    34     0  3840     3     0]
 [    5   151   290    25     3  7060     0]
 [  320    27     0     0     1     0  8581]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('Training Accuracy:',abcl_RF.score(x_train,y_train))
print('Testing Accuracy:',abcl_RF.score(x_test,y_test))

Training Accuracy: 0.9022525510204081
Testing Accuracy: 0.8503273809523809


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.83      0.81      0.82     57141
           2       0.87      0.87      0.87     75000
           3       0.88      0.90      0.89     13131
           4       0.91      0.83      0.87      1797
           5       0.91      0.78      0.84      4468
           6       0.79      0.82      0.81      7534
           7       0.81      0.94      0.87      8929

    accuracy                           0.85    168000
   macro avg       0.86      0.85      0.85    168000
weighted avg       0.85      0.85      0.85    168000

[[46494  8720     1     0    64    20  1842]
 [ 8748 65047   283     1   261   491   169]
 [    3   203 11795   104    10  1016     0]
 [    0     0   220  1486     0    91     0]
 [   45   899    36     0  3472    16     0]
 [    3   192  1108    36     4  6191     0]
 [  500    56     0     0     3     0  8370]]


### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print(gbcl.score(x_train,y_train))
print(gbcl.score(x_test,y_test))

1.0
0.9702738095238095


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.97      0.96      0.97     57141
           2       0.97      0.98      0.97     75000
           3       0.97      0.97      0.97     13131
           4       0.95      0.94      0.95      1797
           5       0.96      0.93      0.94      4468
           6       0.96      0.96      0.96      7534
           7       0.98      0.98      0.98      8929

    accuracy                           0.97    168000
   macro avg       0.97      0.96      0.96    168000
weighted avg       0.97      0.97      0.97    168000

[[55128  1842     1     0    20     6   144]
 [ 1342 73304   102     1   150    76    25]
 [    1    98 12801    60     9   162     0]
 [    0     0    79  1692     0    26     0]
 [   19   280    26     0  4135     8     0]
 [    2    98   206    29     2  7197     0]
 [  163    17     0     0     0     0  8749]]


## Smote Transformed Added Features

Constructing a dataframe with added features and removing features on which some feature engineering has been applied, so as to gauge their performance in classifying cover_types.

In [ ]:
forest_d = forest_c.drop(columns = forest_c.columns[3:10])

In [ ]:
forest_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560000 entries, 0 to 559999
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   Elevation       560000 non-null  float64 
 1   Aspect          560000 non-null  float64 
 2   Slope           560000 non-null  float64 
 3   Hydr_Slope      560000 non-null  float64 
 4   Fire_Hydr       560000 non-null  float64 
 5   Avg_Horizontal  560000 non-null  float64 
 6   Fire_Road       560000 non-null  float64 
 7   Hillshades      560000 non-null  float64 
 8   Hydr_Dist       560000 non-null  float64 
 9   Wilderness      560000 non-null  category
 10  Soil            560000 non-null  category
 11  Cover_Type      560000 non-null  category
dtypes: category(3), float64(9)
memory usage: 41.3 MB


## Modelling on smote transformed added features

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data into train and test in 7:3 ratio

In [ ]:
x = forest_d.drop(columns = ['Cover_Type'])
y = forest_d['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9311326530612245
Testing Score: 0.9000119047619047


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.89      0.89      0.89     57141
           2       0.90      0.91      0.91     75000
           3       0.91      0.93      0.92     13131
           4       0.91      0.89      0.90      1797
           5       0.85      0.84      0.85      4468
           6       0.88      0.86      0.87      7534
           7       0.95      0.94      0.95      8929

    accuracy                           0.90    168000
   macro avg       0.90      0.89      0.90    168000
weighted avg       0.90      0.90      0.90    168000

[[50632  6055     2     0    60    22   370]
 [ 5620 68153   331     2   556   272    66]
 [    1   310 12149   104    28   539     0]
 [    0     0   151  1598     0    48     0]
 [   76   563    42     0  3763    24     0]
 [    9   254   684    60    22  6505     0]
 [  452    73     0     0     2     0  8402]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.451208
1,Aspect,0.045149
2,Slope,0.000987
3,Hydr_Slope,0.002850
4,Fire_Hydr,0.101960
5,Avg_Horizontal,0.059003
6,Fire_Road,0.085741
7,Hillshades,0.028389
8,Hydr_Dist,0.052383
9,Wilderness,0.042226


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9568979591836735
Testing Score: 0.9308214285714286


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.94      0.91      0.92     57141
           2       0.92      0.95      0.94     75000
           3       0.93      0.95      0.94     13131
           4       0.93      0.91      0.92      1797
           5       0.94      0.83      0.88      4468
           6       0.92      0.90      0.91      7534
           7       0.97      0.96      0.96      8929

    accuracy                           0.93    168000
   macro avg       0.93      0.92      0.92    168000
weighted avg       0.93      0.93      0.93    168000

[[51846  5013     1     0    41     7   233]
 [ 2965 71318   252     3   200   224    38]
 [    2   176 12531    79     4   339     0]
 [    0     0   126  1638     0    33     0]
 [   21   676    48     0  3705    18     0]
 [    0   165   529    40     3  6797     0]
 [  347    37     0     0     2     0  8543]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.312762
1,Aspect,0.046572
2,Slope,0.003383
3,Hydr_Slope,0.008432
4,Fire_Hydr,0.085029
5,Avg_Horizontal,0.092105
6,Fire_Road,0.086688
7,Hillshades,0.033169
8,Hydr_Dist,0.050677
9,Wilderness,0.079746


### KNN

*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio


In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_d.drop(columns= ['Cover_Type','Soil','Wilderness'])
y_ = forest_d['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_d[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Slope,Hydr_Slope,Fire_Hydr,Avg_Horizontal,Fire_Road,Hillshades,Hydr_Dist,Soil,Wilderness
0,0.824993,-0.680594,0.015748,1.082570,-0.242679,-0.360650,-0.245514,-0.083351,-1.371248,29.0,1.0
1,1.139856,-1.046790,-1.343958,0.267644,-0.436214,-0.984879,-1.272567,-0.846651,1.663775,22.0,1.0
2,1.084099,1.052138,0.015748,-1.419384,-0.593060,-1.161074,-1.447657,1.106501,1.448859,32.0,3.0
3,0.926668,-0.725252,-0.379393,1.259621,-0.925946,-1.045563,-0.953958,-0.846651,-0.836644,39.0,3.0
4,0.992264,0.382268,-0.379393,-0.645086,-0.699194,-1.022947,-0.971208,1.286101,-0.387040,23.0,1.0


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9009940476190477


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.90      0.89      0.89     57141
           2       0.91      0.91      0.91     75000
           3       0.90      0.91      0.91     13131
           4       0.90      0.89      0.90      1797
           5       0.81      0.83      0.82      4468
           6       0.84      0.86      0.85      7534
           7       0.93      0.95      0.94      8929

    accuracy                           0.90    168000
   macro avg       0.89      0.89      0.89    168000
weighted avg       0.90      0.90      0.90    168000

[[50616  5845     7     0   111    20   542]
 [ 4914 68464   394     3   712   431    82]
 [    4   274 11978   120    36   719     0]
 [    0     1   129  1595     0    72     0]
 [   80   614    48     0  3713    11     2]
 [   23   270   689    48    23  6481     0]
 [  351    55     0     0     3     0  8520]]


### Ensemble Techniques on smote transformed added features

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Score:',bgcl_DT.score(x_train,y_train))
print('Testing Score:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.935094387755102
Training Score: 0.9624744897959183
Testing Score: 0.9360595238095238


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.94      0.92      0.93     57141
           2       0.93      0.95      0.94     75000
           3       0.94      0.95      0.94     13131
           4       0.93      0.92      0.93      1797
           5       0.92      0.87      0.90      4468
           6       0.92      0.91      0.92      7534
           7       0.97      0.96      0.96      8929

    accuracy                           0.94    168000
   macro avg       0.94      0.93      0.93    168000
weighted avg       0.94      0.94      0.94    168000

[[52550  4313     2     0    33     6   237]
 [ 3080 71249   195     1   266   170    39]
 [    0   204 12445    83    13   386     0]
 [    0     0   111  1656     0    30     0]
 [   32   477    40     0  3898    21     0]
 [    3   146   449    39     5  6892     0]
 [  316    43     0     0     2     0  8568]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Score:',bgcl_RF.score(x_train,y_train))
print('Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9179770408163266
Training Score: 0.9384056122448979
Accuracy: 0.9166130952380952


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.93      0.89      0.91     57141
           2       0.91      0.94      0.92     75000
           3       0.91      0.95      0.93     13131
           4       0.92      0.90      0.91      1797
           5       0.93      0.77      0.85      4468
           6       0.90      0.88      0.89      7534
           7       0.96      0.94      0.95      8929

    accuracy                           0.92    168000
   macro avg       0.92      0.90      0.91    168000
weighted avg       0.92      0.92      0.92    168000

[[50802  5985     1     0    46    14   293]
 [ 3497 70674   331     3   197   260    38]
 [    0   244 12417    90     2   378     0]
 [    0     0   147  1623     0    27     0]
 [   28   898    64     0  3458    20     0]
 [    0   220   671    39     3  6601     0]
 [  472    39     0     0     2     0  8416]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Score:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Score:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9001326530612245
Training Score: 0.9997397959183674
Testing Score: 0.9016904761904762


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.91      0.88      0.90     57141
           2       0.91      0.92      0.91     75000
           3       0.90      0.91      0.91     13131
           4       0.91      0.89      0.90      1797
           5       0.81      0.82      0.82      4468
           6       0.84      0.86      0.85      7534
           7       0.93      0.95      0.94      8929

    accuracy                           0.90    168000
   macro avg       0.89      0.89      0.89    168000
weighted avg       0.90      0.90      0.90    168000

[[50464  5977     9     0   105    17   569]
 [ 4639 68779   399     3   682   421    77]
 [    5   272 11990   109    32   723     0]
 [    0     0   133  1592     0    72     0]
 [   77   643    50     0  3678    18     2]
 [   20   258   706    47    26  6477     0]
 [  360    62     0     0     3     0  8504]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

#### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('AdaBoost DT Training Score:',abcl_DT.score(x_train,y_train))
print('AdaBoost DT Testing Score:',abcl_DT.score(x_test,y_test))

AdaBoost DT Training Score: 0.9995841836734694
AdaBoost DT Testing Score: 0.927672619047619


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.93      0.91      0.92     57141
           2       0.92      0.95      0.93     75000
           3       0.94      0.96      0.95     13131
           4       0.96      0.91      0.94      1797
           5       0.97      0.81      0.88      4468
           6       0.95      0.91      0.93      7534
           7       0.98      0.95      0.96      8929

    accuracy                           0.93    168000
   macro avg       0.95      0.91      0.93    168000
weighted avg       0.93      0.93      0.93    168000

[[51724  5278     0     0    10     3   126]
 [ 3627 70882   235     0    99   133    24]
 [    0   197 12661    42     6   225     0]
 [    0     0   126  1642     0    29     0]
 [   24   776    36     0  3626     6     0]
 [    0   183   455    25     1  6870     0]
 [  449    34     0     0     2     0  8444]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('AdaBoost RF Training Score:',abcl_RF.score(x_train,y_train))
print('AdaBoost RF Testing Score:',abcl_RF.score(x_test,y_test))

AdaBoost RF Training Score: 0.8717780612244898
AdaBoost RF Testing Score: 0.8222916666666666


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.84      0.74      0.79     57141
           2       0.88      0.85      0.87     75000
           3       0.84      0.93      0.88     13131
           4       0.88      0.71      0.79      1797
           5       0.64      0.90      0.75      4468
           6       0.84      0.79      0.81      7534
           7       0.54      0.97      0.69      8929

    accuracy                           0.82    168000
   macro avg       0.78      0.84      0.80    168000
weighted avg       0.84      0.82      0.82    168000

[[42100  8066     2     0   559    17  6397]
 [ 7436 63967   444     1  1673   437  1042]
 [   21   169 12217   135    41   548     0]
 [    0     0   441  1284     0    72     0]
 [   83   297    44     0  4026    18     0]
 [   30   146  1374    35    24  5925     0]
 [  285    13     0     0     5     0  8626]]


### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print(gbcl.score(x_train,y_train))
print(gbcl.score(x_test,y_test))

1.0
0.9670892857142858


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.97      0.96      0.97     57141
           2       0.97      0.97      0.97     75000
           3       0.96      0.97      0.97     13131
           4       0.95      0.94      0.95      1797
           5       0.95      0.92      0.93      4468
           6       0.96      0.95      0.95      7534
           7       0.98      0.98      0.98      8929

    accuracy                           0.97    168000
   macro avg       0.96      0.96      0.96    168000
weighted avg       0.97      0.97      0.97    168000

[[54927  2029     3     0    24    11   147]
 [ 1471 73112   123     1   169    99    25]
 [    0   114 12760    64     9   184     0]
 [    0     0    78  1695     0    24     0]
 [   22   316    24     0  4102     4     0]
 [    2   100   254    24     2  7152     0]
 [  178    26     0     0     2     0  8723]]


## Smote-Transformed selected Features

Selecting features on basis of their importance in decision tree and random forest models. Also, considering the overall performance of the models.

In [ ]:
forest_f = forest_c.drop(columns = forest_c.columns[2:5])
forest_f = forest_f.drop(columns = forest_c.columns[7:9])
forest_f = forest_f.drop(columns = forest_c.columns[10])
forest_f = forest_f.drop(columns = forest_c.columns[14])

In [ ]:
forest_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560000 entries, 0 to 559999
Data columns (total 12 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Elevation                           560000 non-null  float64 
 1   Aspect                              560000 non-null  float64 
 2   Horizontal_Distance_To_Roadways     560000 non-null  float64 
 3   Hillshade_9am                       560000 non-null  float64 
 4   Horizontal_Distance_To_Fire_Points  560000 non-null  float64 
 5   Fire_Hydr                           560000 non-null  float64 
 6   Avg_Horizontal                      560000 non-null  float64 
 7   Fire_Road                           560000 non-null  float64 
 8   Hydr_Dist                           560000 non-null  float64 
 9   Wilderness                          560000 non-null  category
 10  Soil                                560000 non-null  category
 11  Cover_Type   

## Modelling on selected features

### Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

Dividing data into x (independent variable) and y (dependent variable) and then splitting data into train and test in 7:3 ratio

In [ ]:
x = forest_f.drop(columns = ['Cover_Type'])
y = forest_f['Cover_Type']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [ ]:
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [ ]:
print('Training Score:',dt.score(x_train, y_train))
print('Testing Score:',dt.score(x_test, y_test))

Training Score: 0.9358571428571428
Testing Score: 0.907


In [ ]:
y_pred = dt.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.91      0.89      0.90     57141
           2       0.91      0.92      0.91     75000
           3       0.91      0.92      0.92     13131
           4       0.91      0.88      0.89      1797
           5       0.85      0.84      0.85      4468
           6       0.88      0.88      0.88      7534
           7       0.94      0.95      0.95      8929

    accuracy                           0.91    168000
   macro avg       0.90      0.90      0.90    168000
weighted avg       0.91      0.91      0.91    168000

[[50772  5817     2     0    69    19   462]
 [ 4767 69116   288     1   520   251    57]
 [    7   351 12064   104    41   564     0]
 [    0     4   154  1584     0    55     0]
 [   73   553    51     0  3760    31     0]
 [   10   219   627    54    22  6602     0]
 [  389    60     0     0     2     0  8478]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.443170
1,Aspect,0.035527
2,Horizontal_Distance_To_Roadways,0.076752
3,Hillshade_9am,0.032482
4,Horizontal_Distance_To_Fire_Points,0.054285
5,Fire_Hydr,0.047462
6,Avg_Horizontal,0.038130
7,Fire_Road,0.064403
8,Hydr_Dist,0.047827
9,Wilderness,0.042145


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [ ]:
print('Training Score:',rforest.score(x_train, y_train))
print('Testing Score:',rforest.score(x_test, y_test))

Training Score: 0.9672219387755102
Testing Score: 0.9461845238095238


In [ ]:
y_pred = rforest.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     57141
           2       0.94      0.96      0.95     75000
           3       0.94      0.96      0.95     13131
           4       0.94      0.93      0.94      1797
           5       0.94      0.87      0.90      4468
           6       0.94      0.92      0.93      7534
           7       0.97      0.96      0.97      8929

    accuracy                           0.95    168000
   macro avg       0.95      0.93      0.94    168000
weighted avg       0.95      0.95      0.95    168000

[[53098  3795     1     0    39     5   203]
 [ 2257 72153   207     2   207   137    37]
 [    0   149 12619    63     8   292     0]
 [    0     0    93  1673     0    31     0]
 [   17   519    41     0  3880    11     0]
 [    0   131   442    35     4  6922     0]
 [  276    36     0     0     3     0  8614]]


In [ ]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.293245
1,Aspect,0.037148
2,Horizontal_Distance_To_Roadways,0.071888
3,Hillshade_9am,0.031676
4,Horizontal_Distance_To_Fire_Points,0.052535
5,Fire_Hydr,0.055029
6,Avg_Horizontal,0.073451
7,Fire_Road,0.069435
8,Hydr_Dist,0.045413
9,Wilderness,0.075373


### KNN


*   scaling all numerical columns and saving it in a dataframe 'x_std'
*   concatenating all the independent data into a dataframe 'x_' 
*   dividing data to x_ (independent variable) and y_ (dependent variable)
*   splitting data into train and test in 7:3 ratio

In [ ]:
from sklearn.preprocessing import StandardScaler

x1 = forest_f.drop(columns= ['Cover_Type','Soil','Wilderness'])
y_ = forest_f['Cover_Type']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x_ = pd.concat([x_std,forest_f[['Soil','Wilderness']]], axis = 1)
x_.head()

,Elevation,Aspect,Horizontal_Distance_To_Roadways,Hillshade_9am,Horizontal_Distance_To_Fire_Points,Fire_Hydr,Avg_Horizontal,Fire_Road,Hydr_Dist,Soil,Wilderness
0,0.824993,-0.680594,-0.242203,0.806948,-0.031070,-0.242679,-0.360650,-0.245514,-1.371248,29.0,1.0
1,1.139856,-1.046790,-1.089579,0.259169,-0.884791,-0.436214,-0.984879,-1.272567,1.663775,22.0,1.0
2,1.084099,1.052138,-1.232150,-0.726835,-1.032981,-0.593060,-1.161074,-1.447657,1.448859,32.0,3.0
3,0.926668,-0.725252,-0.708370,0.953023,-0.758116,-0.925946,-1.045563,-0.953958,-0.836644,39.0,3.0
4,0.992264,0.382268,-0.873029,0.003538,-0.598972,-0.699194,-1.022947,-0.971208,-0.387040,23.0,1.0


In [ ]:
x_train_k, x_test_k, y_train_k, y_test_k = train_test_split(x_, y_, test_size = .3, random_state=25)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train_k,y_train_k)
y_pred_k = knn.predict(x_test_k)

In [ ]:
print('Training Score:',knn.score(x_train_k,y_train_k))
print('Testing Score:',knn.score(x_test_k,y_test_k))

Training Score: 1.0
Testing Score: 0.9617559523809524


In [ ]:
print(classification_report(y_test_k, y_pred_k))
print(confusion_matrix(y_test_k, y_pred_k))

              precision    recall  f1-score   support

           1       0.96      0.96      0.96     57141
           2       0.96      0.96      0.96     75000
           3       0.97      0.97      0.97     13131
           4       0.95      0.95      0.95      1797
           5       0.91      0.95      0.93      4468
           6       0.95      0.96      0.95      7534
           7       0.97      0.98      0.97      8929

    accuracy                           0.96    168000
   macro avg       0.95      0.96      0.96    168000
weighted avg       0.96      0.96      0.96    168000

[[54589  2243     1     0    52     9   247]
 [ 2005 72369   140     1   321   124    40]
 [    1    97 12710    62    21   240     0]
 [    0     0    61  1701     0    35     0]
 [   19   193    24     0  4227     4     1]
 [   10    84   187    31    10  7212     0]
 [  149    12     0     0     1     0  8767]]


### Ensemble Techniques on selected features

In [ ]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

#### Decision Bag

In [ ]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT,n_estimators=51,oob_score=True,random_state=25)
bgcl_DT.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [ ]:
print('OOB SCORE:',bgcl_DT.oob_score_)
print('Training Accuracy:',bgcl_DT.score(x_train,y_train))
print('Testing Accuracy:',bgcl_DT.score(x_test,y_test))

OOB SCORE: 0.9427142857142857
Training Accuracy: 0.9671224489795919
Testing Accuracy: 0.9439285714285715


In [ ]:
print(classification_report(y_test,bgcl_DT.predict(x_test)))
print(confusion_matrix(y_test,bgcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     57141
           2       0.94      0.96      0.95     75000
           3       0.95      0.96      0.95     13131
           4       0.94      0.92      0.93      1797
           5       0.92      0.87      0.89      4468
           6       0.93      0.93      0.93      7534
           7       0.97      0.96      0.96      8929

    accuracy                           0.94    168000
   macro avg       0.94      0.93      0.94    168000
weighted avg       0.94      0.94      0.94    168000

[[53304  3552     2     0    45     6   232]
 [ 2765 71645   164     1   262   125    38]
 [    0   181 12554    65    12   319     0]
 [    0     2   107  1653     0    35     0]
 [   35   500    41     0  3873    19     0]
 [    2   128   384    38     5  6977     0]
 [  300    53     0     0     2     0  8574]]


#### Random Forest Bag

In [ ]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF,n_estimators=51,oob_score=True,random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [ ]:
print('OOB SCORE:',bgcl_RF.oob_score_)
print('Training Accuracy:',bgcl_RF.score(x_train,y_train))
print('Testing Accuracy:',bgcl_RF.score(x_test,y_test))

OOB SCORE: 0.9362295918367347
Training Accuracy: 0.9529362244897959
Testing Accuracy: 0.9353630952380952


In [ ]:
print(classification_report(y_test,bgcl_RF.predict(x_test)))
print(confusion_matrix(y_test,bgcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.95      0.91      0.93     57141
           2       0.93      0.96      0.94     75000
           3       0.93      0.96      0.94     13131
           4       0.94      0.92      0.93      1797
           5       0.93      0.82      0.87      4468
           6       0.92      0.90      0.91      7534
           7       0.97      0.96      0.96      8929

    accuracy                           0.94    168000
   macro avg       0.94      0.92      0.93    168000
weighted avg       0.94      0.94      0.94    168000

[[52264  4569     2     0    49     9   248]
 [ 2590 71711   261     1   222   175    40]
 [    0   193 12543    73     6   316     0]
 [    0     0   114  1650     0    33     0]
 [   20   700    64     0  3667    17     0]
 [    0   176   565    35     3  6755     0]
 [  336    40     0     0     2     0  8551]]


#### KNN Bag

In [ ]:
bgcl_KN = BaggingClassifier(base_estimator=base_KNN,n_estimators=51,oob_score=True,random_state=25)
bgcl_KN.fit(x_train_k, y_train_k)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [ ]:
print('OOB SCORE:',bgcl_KN.oob_score_)
print('Training Accuracy:',bgcl_KN.score(x_train_k,y_train_k))
print('Testing Accuracy:',bgcl_KN.score(x_test_k,y_test_k))

OOB SCORE: 0.9611454081632653
Training Accuracy: 0.9998724489795918
Testing Accuracy: 0.9611904761904762


In [ ]:
print(classification_report(y_test_k,bgcl_KN.predict(x_test_k)))
print(confusion_matrix(y_test_k,bgcl_KN.predict(x_test_k)))

              precision    recall  f1-score   support

           1       0.96      0.95      0.96     57141
           2       0.96      0.97      0.96     75000
           3       0.97      0.97      0.97     13131
           4       0.94      0.94      0.94      1797
           5       0.91      0.94      0.93      4468
           6       0.94      0.96      0.95      7534
           7       0.97      0.98      0.97      8929

    accuracy                           0.96    168000
   macro avg       0.95      0.96      0.95    168000
weighted avg       0.96      0.96      0.96    168000

[[54519  2299     0     0    55     8   260]
 [ 1960 72390   143     1   325   137    44]
 [    1    97 12680    69    19   265     0]
 [    0     0    68  1690     0    39     0]
 [   19   196    26     0  4222     4     1]
 [    8    84   190    32     8  7212     0]
 [  148    13     0     0     1     0  8767]]


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

##### Decision Tree

In [ ]:
abcl_DT.fit(x_train,y_train)
print('Training Accuracy:',abcl_DT.score(x_train,y_train))
print('Testing Accuracy:',abcl_DT.score(x_test,y_test))

Training Accuracy: 0.9998724489795918
Testing Accuracy: 0.9660892857142858


In [ ]:
print(classification_report(y_test,abcl_DT.predict(x_test)))
print(confusion_matrix(y_test,abcl_DT.predict(x_test)))

              precision    recall  f1-score   support

           1       0.97      0.96      0.96     57141
           2       0.96      0.97      0.97     75000
           3       0.97      0.98      0.97     13131
           4       0.96      0.95      0.96      1797
           5       0.97      0.92      0.95      4468
           6       0.97      0.96      0.96      7534
           7       0.98      0.98      0.98      8929

    accuracy                           0.97    168000
   macro avg       0.97      0.96      0.96    168000
weighted avg       0.97      0.97      0.97    168000

[[54630  2369     0     0    16     5   121]
 [ 1666 73035   101     0    98    76    24]
 [    0    95 12845    44     3   144     0]
 [    0     0    71  1708     0    18     0]
 [   16   316    20     0  4112     4     0]
 [    0    83   199    25     3  7224     0]
 [  163    15     0     0     2     0  8749]]


#### Random Forest

In [ ]:
abcl_RF.fit(x_train,y_train)
print('Training Accuracy:',abcl_RF.score(x_train,y_train))
print('Testing Accuracy:',abcl_RF.score(x_test,y_test))

Training Accuracy: 0.6251556122448979
Testing Accuracy: 0.6043333333333333


In [ ]:
print(classification_report(y_test,abcl_RF.predict(x_test)))
print(confusion_matrix(y_test,abcl_RF.predict(x_test)))

              precision    recall  f1-score   support

           1       0.48      1.00      0.65     57141
           2       0.88      0.33      0.48     75000
           3       0.93      0.88      0.90     13131
           4       0.95      0.91      0.93      1797
           5       0.99      0.14      0.24      4468
           6       0.95      0.71      0.81      7534
           7       1.00      0.03      0.07      8929

    accuracy                           0.60    168000
   macro avg       0.88      0.57      0.58    168000
weighted avg       0.76      0.60      0.56    168000

[[57123    18     0     0     0     0     0]
 [49914 25007    61     0     0    18     0]
 [  235  1125 11494    54     4   219     0]
 [    0    17   114  1636     0    30     0]
 [ 2639  1208    13     0   608     0     0]
 [  613   925   616    30     0  5350     0]
 [ 8592    27     0     0     0     0   310]]


### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
print('Training Score:',gbcl.score(x_train,y_train))
print('Testing Score:',gbcl.score(x_test,y_test))

Training Score: 1.0
Testing Score: 0.9724702380952381


In [ ]:
print(classification_report(y_test,gbcl.predict(x_test)))
print(confusion_matrix(y_test,gbcl.predict(x_test)))

              precision    recall  f1-score   support

           1       0.98      0.97      0.97     57141
           2       0.97      0.98      0.98     75000
           3       0.97      0.97      0.97     13131
           4       0.95      0.94      0.94      1797
           5       0.96      0.94      0.95      4468
           6       0.96      0.96      0.96      7534
           7       0.98      0.98      0.98      8929

    accuracy                           0.97    168000
   macro avg       0.97      0.96      0.96    168000
weighted avg       0.97      0.97      0.97    168000

[[55350  1624     1     0    21     8   137]
 [ 1250 73386   101     0   163    72    28]
 [    0   104 12785    67     9   166     0]
 [    0     0    73  1693     0    31     0]
 [   14   237    17     0  4193     7     0]
 [    1    80   214    28     1  7210     0]
 [  150    19     0     0     2     0  8758]]
